## FEC Campaign Finance

### Committee Master

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime as dt

In [2]:
year = '2018'

In [3]:
# read in & clean
cm = pd.read_csv(
    f'https://www.fec.gov/files/bulk-downloads/{year}/cm{year[2:]}.zip', 
    sep = '|', 
    error_bad_lines = False, 
    header = None, 
)
print('original length:', len(cm))

# add column headers from separate file
headers = pd.read_csv(
    'https://www.fec.gov/files/bulk-downloads/data_dictionaries/cm_header_file.csv', 
)
cm.columns = [x.lower() for x in headers.columns]

# preserve uncleaned tres_nm & cmte_st1
cm['orig_tres_nm'] = list(cm['tres_nm'].values)
cm['orig_cmte_st1'] = list(cm['cmte_st1'].values)

# prep strings for deduping
change_dict = {
    ' STREET' : ' ST',
    'AVENUE' : 'AVE',
    ' DRIVE' : ' DR',
    ' ROAD' : ' RD',
    ' SUITE' : ' STE',
    '.' : '',
    ',' : '',
    'BOULEVARD' : 'BLVD',
    ' PLACE' : ' PL',
    
}
keys = list(set(change_dict.keys()))
for key in keys:
    cm['cmte_st1'] = [str(x).replace(key, change_dict[key]) for x in cm['cmte_st1']]

keep_0 = [
    ' STE ',
    '#',
    ' ST ',
    ' AVE ',
    ' RD ',
    ' PL ', 
    ' BLVD ', 
    ' DR ',
]

cm['cmte_st1'] = [x.split(' STE ')[0].split('#')[0].split(' NUM ')[0] for \
                  x in cm['cmte_st1']]
cm['cmte_st1'] = [' '.join(str(x).split()) for x in cm['cmte_st1']]

cm['cmte_zip'] = [str(x).strip() for x in cm['cmte_zip']]
cm.loc[cm['cmte_zip'].apply(lambda x: len(str(x))) == 9, 'cmte_zip'] = \
cm.loc[cm['cmte_zip'].apply(lambda x: len(str(x))) == 9, 'cmte_zip'].apply(lambda x: x[:5])

cm['cmte_zip'].apply(lambda x: len(x)).value_counts().sort_index()

original length: 19016


3       33
4        8
5    18971
6        1
8        3
Name: cmte_zip, dtype: int64

In [4]:
cm.loc[cm['cmte_zip'].apply(lambda x: len(str(x))) != 5, 
       ['cmte_st1', 'cmte_city', 'cmte_st', 'cmte_zip']].sort_values([
    'cmte_zip', 
    'cmte_st1', 
    'cmte_city',
    'cmte_st',
])

,cmte_st1,cmte_city,cmte_st,cmte_zip
10616,PO BOX 2443,COVINGTON,LA,04342443
18179,510A SHERMAN ST,CANTON,MA,2021
18188,580 BRIDGE ST,DEDHAM,MA,2026
18189,580 BRIDGE ST,DEDHAM,MA,2026
18183,17 PLEASANT HILL AVE,BOSTON,MA,21232813
18187,33 THORONDIKE ST,BROOKLINE,MA,2446
895,5 BROAD ST,SUMTER,SC,291501
18195,393 DORCHESTER RD,LYME,NH,3768
2060,1210 CORBIN ST,ELIZABETH,NJ,7201
10840,847 KENSINGTON CV,SPRINGDALE,AR,7276


In [5]:
fix_zip = {
    '291501' : '29150',
    '89053145' : '89015',
    '04342443' : '70433',
    '21232813' : '02126',
    '7276' : '72762',
    '2021' : '02021',
    '2026' : '02026',
    '2446' : '02446',
    '3768' : '03768',
    '7201' : '07201',
    '7724' : '07724',
    
}
for key in list(fix_zip.keys()):
    cm.loc[cm['cmte_zip'] == key, 'cmte_zip'] = fix_zip[key]

cm.loc[cm['cmte_st1'] == '1055 CATALPA RD', 'cmte_zip'] = '91007'
cm.loc[cm['cmte_st1'] == '2008 OLDTOWN VALLEY RD SE', 'cmte_zip'] = '44663'
cm.loc[
    (cm['cmte_st1'] == '247 THIRD ST') & \
    (cm['cmte_city'] == 'ASHLAND'), 
    'cmte_zip'] = '97520'
cm.loc[cm['cmte_st1'] == '420 KEWANNA DR', 'cmte_zip'] = '47130'
cm.loc[cm['cmte_st1'] == '868 CHURCH ST -', 'cmte_zip'] = '11716'
cm.loc[
    (cm['cmte_st1'] == 'PO BOX 334') & \
    (cm['cmte_city'] == 'DAVIDSONVILLE'), 
    'cmte_zip'] = '21035'
cm.loc[
    (cm['cmte_st1'] == 'PO BOX 356') & \
    (cm['cmte_city'] == 'LIVINGSTON'), 
    'cmte_zip'] = '07039'

In [6]:
cm.loc[cm['cmte_zip'].apply(lambda x: len(x) == 3), 'cmte_zip'] = np.nan
# create full address column from cleaned parts
cm['address'] = \
cm['cmte_st1'].apply(lambda x: str(x).strip()) + ', ' + \
cm['cmte_city'].apply(lambda x: str(x).strip()) + ', ' + \
cm['cmte_st'].apply(lambda x: str(x).strip()) + ' ' + \
cm['cmte_zip'].apply(lambda x: str(x).strip())

cm.head(2)

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
0,C00000018,IRONWORKERS LOCAL UNION NO. 25 POLITICAL EDUCA...,STEVEN N GULICK,43335 W 10 MILE,P O BOX 965,NOVI,MI,48050,U,Q,NaN,T,NaN,IRON WORKERS; INT'L ASS'N OF BRIDGE...,H8TX22313,STEVEN N GULICK,43335 W 10 MILE,"43335 W 10 MILE, NOVI, MI 48050"
1,C00000059,HALLMARK CARDS PAC,SARAH MOE,2501 MCGEE,MD #500,KANSAS CITY,MO,64108,U,Q,UNK,M,C,NaN,NaN,SARAH MOE,2501 MCGEE,"2501 MCGEE, KANSAS CITY, MO 64108"


In [7]:
# check for duplicate rows
cm[cm.duplicated(keep = False)]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
13167,C00654699,THE COMMITTEE TO ELECT JUAN PARKS,"MCMILLAN, EVA MRS",6067 FAIRINGTON FARMS LANE,NaN,LITHONIA,GA,30038,P,H,DEM,Q,NaN,NONE,H8GA04117,"MCMILLAN, EVA MRS",6067 FAIRINGTON FARMS LANE,"6067 FAIRINGTON FARMS LANE, LITHONIA, GA 30038"
13168,C00654699,THE COMMITTEE TO ELECT JUAN PARKS,"MCMILLAN, EVA MRS",6067 FAIRINGTON FARMS LANE,NaN,LITHONIA,GA,30038,P,H,DEM,Q,NaN,NONE,H8GA04117,"MCMILLAN, EVA MRS",6067 FAIRINGTON FARMS LANE,"6067 FAIRINGTON FARMS LANE, LITHONIA, GA 30038"
16591,C00690123,INDEPENDENT VOICES OF CHANGE,"PHELAN, JOHN",2 MEGHANS WAY,NaN,LYNN,MA,01904,U,O,NaN,Q,NaN,NONE,H8NJ07256,"PHELAN, JOHN",2 MEGHANS WAY,"2 MEGHANS WAY, LYNN, MA 01904"
16592,C00690123,INDEPENDENT VOICES OF CHANGE,"PHELAN, JOHN",2 MEGHANS WAY,NaN,LYNN,MA,01904,U,O,NaN,Q,NaN,NONE,H8NJ07256,"PHELAN, JOHN",2 MEGHANS WAY,"2 MEGHANS WAY, LYNN, MA 01904"


In [8]:
# drop duplicates
cm.drop_duplicates(keep = 'first', inplace = True)

In [9]:
# see which cmte_ids appear more than once
cm['cmte_id'].value_counts()[cm['cmte_id'].value_counts() > 1]

Series([], Name: cmte_id, dtype: int64)

In [10]:
# check for nulls
cm.isnull().sum()[cm.isnull().sum() > 0].sort_values()

cmte_nm                    10
cmte_tp                    11
cmte_dsgn                  13
cmte_st                    20
cmte_city                  22
cmte_zip                   26
orig_cmte_st1              30
tres_nm                  1519
orig_tres_nm             1519
cmte_pty_affiliation    11644
connected_org_nm        12034
cand_id                 12333
cmte_st2                15299
org_tp                  15421
dtype: int64

In [11]:
# see which have missing cmte_nm
cm[cm['cmte_nm'].isnull()]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
8006,C00575308,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,"nan, nan, nan nan"
8082,C00576454,NaN,"PENN, MARGIE NELL",3401 WILCOX BLVD,NaN,CHATTANOOGA,TN,37411,A,P,DEM,A,NaN,NaN,P60006921,"PENN, MARGIE NELL",3401 WILCOX BLVD,"3401 WILCOX BLVD, CHATTANOOGA, TN 37411"
10164,C00616649,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,"nan, nan, nan nan"
10419,C00619916,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,"nan, nan, nan nan"
10519,C00621201,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,"nan, nan, nan nan"
12322,C00641233,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,"nan, nan, nan nan"
13088,C00653873,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,"nan, nan, nan nan"
13089,C00653881,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,"nan, nan, nan nan"
13396,C00657031,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,"nan, nan, nan nan"
16999,C30001184,NaN,NaN,PO BOX 33524,NaN,WASHINGTON,DC,20033,U,E,NaN,A,NaN,NaN,NaN,NaN,PO BOX 33524,"PO BOX 33524, WASHINGTON, DC 20033"


In [12]:
# fill in placeholder cmte_nm
cm.loc[cm['cmte_nm'].isnull(), 'cmte_nm'] = '(cmte_nm missing)'

In [13]:
# see which have missing address
# then see if any other rows have matching cmte_nm
cm[cm['address'].isnull()]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address


In [14]:
# see which street addresses appear most frequently
cm['address'].value_counts()[:20]

228 S WASHINGTON ST, ALEXANDRIA, VA 22314        202
918 PENNSYLVANIA AVE SE, WASHINGTON, DC 20003    118
824 S MILLEDGE AVE, ATHENS, GA 30605             108
PO BOX 26141, ALEXANDRIA, VA 22313               102
700 13TH ST NW, WASHINGTON, DC 20005              67
PO BOX 9891, ARLINGTON, VA 22219                  57
610 S BLVD, TAMPA, FL 33606                       55
611 PENNSYLVANIA AVE SE, WASHINGTON, DC 20003     52
777 S FIGUEROA ST, LOS ANGELES, CA 90017          43
PO BOX 30844, BETHESDA, MD 20824                  42
5429 MADISON AVE, SACRAMENTO, CA 95841            37
PO BOX 15320, WASHINGTON, DC 20003                36
120 MARYLAND AVE NE, WASHINGTON, DC 20002         36
910 17TH ST NW, WASHINGTON, DC 20006              35
525 E SEASIDE WAY, LONG BEACH, CA 90802           30
555 CAPITOL MALL, SACRAMENTO, CA 95814            29
249 E OCEAN BLVD, LONG BEACH, CA 90802            29
PO BOX 75357, WASHINGTON, DC 20013                28
901 N WASHINGTON ST, ALEXANDRIA, VA 22314     

In [15]:
# fill in placeholder for missing treasurer name 
print(len(cm[cm['tres_nm'].isnull()]))
cm.loc[cm['tres_nm'].isnull(), 'tres_nm'] = '(no treasurer listed)'
cm.loc[cm['tres_nm'].apply(lambda x: x in ['.', 'NONE', '']), 'tres_nm'] = '(no treasurer listed)'

# convert to <first name> <last name> format from <last name>, <first name>
cm['tres_nm'] = [' '.join([' '.join(x.split(', ')[1:]), x.split(', ')[0]]) if ', ' in \
                 x else x for x in cm['tres_nm']]
# clean up spacing
# looks like this could just be ' '.join(x.strip()) 
# but i feel like i did it this way for a reason... leaving it
cm['tres_nm'] = [' '.join([x for x in list(x.split(' ')) if len(x) > 1]).strip() for \
                 x in cm['tres_nm']]

# prep for deduping

# substitutions
subs = {
    'XAVUER' : 'XAVIER',
    'JSOEPH' : 'JOSEPH',
    'WILLAIM' : 'WILLIAM',
    'V?RONIQUE' : 'VERONIQUE',
    '.' : '',
    ',' : '',
    
}
for key in list(subs.keys()):
    cm['tres_nm'] = [x.replace(key, subs[key]) for x in cm['tres_nm']]

# drops
drops = [
    'MRS',
    'MS',
    'MR',
    'MISS',
    'JR',
    'SR',
    'UNDEFINED',
    'JD',
    'CPA',
    'HONORABLE',
    'FED',
    'II', 
    'III',
    'IV',
    'ESQ',
    'DR',
    'MD',
    'DO',
    'CFO',
    'CEO',
    'CTO',
    'TREAS',
    'US',
    '1970',
    '--SELECT',
    '1973',
    '1979',
    
]
for droplet in drops:
    cm['tres_nm'] = [' '.join([x for x in x.split() if \
                               ((x not in drops) & (len(x) > 1))]) for x in cm['tres_nm']]

# a bunch of entries have last name appearing twice; 
# check for this and keep only first instance
cm['tres_nm'] = [x.split() for x in cm['tres_nm']]
def f7(seq):
    '''https://stackoverflow.com/questions/480214/\
    how-do-you-remove-duplicates-from-a-list-whilst-preserving-order'''
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]            
cm['tres_nm'] = [' '.join(f7(x)) for x in cm['tres_nm']]

# see tres_nms appearing most frequently
cm['tres_nm'].value_counts()[:20]

1519


(no treasurer listed)    1526
PAUL KILGORE              171
CHRIS MARSTON             126
JUDITH ZAMORE             122
LISA LISKER               116
JENNIFER MAY               88
BRADLEY CRATE              75
JAY PETTERSON              75
NANCY WATKINS              69
KEITH DAVIS                66
CABELL HOBBS               66
DAVID SATTERFIELD          61
BENJAMIN OTTENHOFF         56
JANICA KYRIACOPOULOS       49
THOMAS DATWYLER            40
STEVEN MELE                35
GARY CRUMMITT              34
JEN SLATER                 31
KELLY LAWLER               31
STEVEN MARTIN              29
Name: tres_nm, dtype: int64

In [16]:
# what is street address of committees not listing treasurer
cm.loc[cm['tres_nm'] == '(no treasurer listed)', 'address'].value_counts()[:20]

nan, nan, nan nan                                 17
1201 CONNECTICUT AVE NW, WASHINGTON, DC 20036     10
700 13TH ST NW, WASHINGTON, DC 20005               6
815 16TH ST NW, WASHINGTON, DC 20006               5
300 M ST SE, WASHINGTON, DC 20003                  5
555 CAPITOL MALL, SACRAMENTO, CA 95814             5
PO BOX 2259, WILMINGTON, NC 28402                  4
1401 NEW YORK AVE NW, WASHINGTON, DC 20005         4
5429 MADISON AVE, SACRAMENTO, CA 95841             4
1120 CONNECTICUT AVE NW, WASHINGTON, DC 20036      4
1747 PENNSYLVANIA AVE NW, WASHINGTON, DC 20006     4
50 F ST NW, WASHINGTON, DC 20001                   3
1201 15TH ST NW, WASHINGTON, DC 20005              3
888 16TH ST NW, WASHINGTON, DC 20006               3
625 BROAD ST, NEWARK, NJ 07102                     3
705-2 E BIDWELL ST, FOLSOM, CA 95630               3
905 16TH ST NW, WASHINGTON, DC 20006               3
1201 F ST NW, WASHINGTON, DC 20004                 3
1150 GRAND BLVD, KANSAS CITY, MO 64106        

In [17]:
list(cm.loc[cm['address'] == '1201 CONNECTICUT AVE NW, WASHINGTON, DC 20036', 'cmte_nm'])

['CONSTITUTIONAL RESPONSIBILITY PROJECT',
 'SOCAL HEALTH CARE COALITION A PROJECT OF SIXTEEN THIRTY FUND',
 'FLORIDIANS FOR A FAIR SHAKE A PROJECT OF SIXTEEN THIRTY FUND',
 'DEMAND JUSTICE A PROJECT OF SIXTEEN THIRTY FUND',
 'SIXTEEN THIRTY FUND / MAKE IT WORK AMERICA1',
 'SIXTEEN THIRTY FUND/MAKE IT WORK ACTION',
 'SIXTEEN THIRTY FUND/NOT ONE PENNY',
 'FLORIDIANS FOR A FAIR SHAKE A PROJECT OF SIXTEEN THIRTY FUND',
 'SOCAL HEALTH CARE COALITION A PROJECT OF SIXTEEN THIRTY FUND',
 'OHIOANS FOR ECONOMIC OPPORTUNITY A PROJECT OF SIXTEEN THIRTY FUND']

In [18]:
list(cm.loc[
    (cm['address'] == '700 13TH ST NW, WASHINGTON, DC 20005') & \
    (cm['tres_nm'] == '(no treasurer listed)'), 'cmte_nm'])

['MAJORITY FORWARD',
 'MAJORITY FORWARD',
 'AMERICA WORKING TOGETHER',
 'DUTY AND HONOR',
 'BLACK PROGRESSIVE ACTION COALITION',
 'BLACK ECONOMIC ALLIANCE FUND']

In [19]:
list(cm.loc[
    (cm['address'] == '555 CAPITOL MALL, SACRAMENTO, CA 95814') & \
    (cm['tres_nm'] == '(no treasurer listed)'), 'cmte_nm'])

['PLANNED PARENTHOOD ADVOCACY PROJECT LOS ANGELES COUNTY',
 'PLANNED PARENTHOOD AFFILIATES OF CALIF',
 'ALAMEDANS UNITED SUPPORTING VELLA AND ASHCRAFT FOR CITY COUNCIL, ET AL',
 'OPPORTUNITY PAC - A COALITION OF TEACHERS HEALTH CARE GIVERS FACULTY MEMBERS SCHOOL EMPLOYEES AND PUBLIC AND PR',
 'GOLDEN CALIFORNIA COMMITTEE SPONSORED BY THE SEIU CALIFORNIA STATE COUNCIL']

### Dedupe treasurer names

This is a journey.

In [20]:
def findall(name):
    '''find pattern in treasurer names columns'''
    all_tres = list(set(cm['tres_nm']))
    if type(name) == str:
        these = [x for x in all_tres if name.lower() in str(x).lower()]
    else:  
        these = all_tres
        for which in name:
            these = [x for x in these if which in str(x).lower()]
    return sorted(these)

In [21]:
def getall(these, new):
    '''assign deduped treasurer name'''
    print('orig len:', len(cm[cm['tres_nm'] == new]))
    for which in these:
        cm.loc[cm['tres_nm'] == which, 'tres_nm'] = new
    print('new len:', len(cm[cm['tres_nm'] == new]))
    return 

In [22]:
# this list keeps track of the ones we've already addressed
dones = ['(no treasurer listed)']

In [23]:
findall(' kil')

['CLIFTON JULIUS KILBY',
 'EDWIN JAY KILPATRICK',
 'GERALD KILPATRICK',
 'JEFFREY TUTTLE KILLEEN',
 'KEVIN KILEY',
 'LAURA KILGORE MCMENAMIN',
 'LAURA KILMER',
 'LAWRENCE KILGORE',
 'MEAGHAN KILLION JOYCE',
 'MILES KILCOIN',
 'PAUL KILGORE',
 'RACHEL KILPATRICK',
 'ROBERT DONALD KILPATRICK',
 'ROBIN KILEY',
 'TERRY KILROY',
 'WILLIAM KILLMER']

In [24]:
findall(' joyce')

['ELAINE JOYCE MCKINNEY',
 'JONATHAN JOYCE',
 'KATHY JOYCE STREUSAND',
 'MEAGHAN KILLION JOYCE',
 'STEPHEN JOYCE']

In [25]:
dones.extend(findall(' kil'))

In [26]:
findall(' lis')

['(no treasurer listed)',
 'ANTHONY LISTROM',
 'EVAN ROGER LISTOPAD',
 'JOHN LISTAK',
 'LISA LISKER',
 'SUSIE LISA']

In [27]:
dones.extend(findall(' lis'))

In [28]:
findall('lisker')

['LISA LISKER', 'LISKERLISA']

In [29]:
new = 'LISA LISKER'
dones.append(new)

these = findall('lisker')

getall(these, new)

orig len: 116
new len: 117


In [30]:
findall(['c', 'marst'])

['CHRIS MARSTEN', 'CHRIS MARSTON', 'CHRISTOPHER MARSTON']

In [31]:
cm[cm['tres_nm'] == 'CHRIS MARSTEN']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
11508,C00632760,WHITE COAT WASTE PAC,CHRIS MARSTEN,PO BOX 26141,NaN,ALEXANDRIA,VA,22313,U,Q,NaN,Q,NaN,NaN,NaN,CHRIS MARSTEN,PO BOX 26141,"PO BOX 26141, ALEXANDRIA, VA 22313"
12366,C00646463,BUCKEYE BATTLE PAC,CHRIS MARSTEN,PO BOX 26141,NaN,ALEXANDRIA,VA,22313,U,O,NaN,T,NaN,NaN,NaN,CHRIS MARSTEN,PO BOX 26141,"PO BOX 26141, ALEXANDRIA, VA 22313"
14309,C00666511,WE STAND FOR BETTER,CHRIS MARSTEN,PO BOX 26141,NaN,ALEXANDRIA,VA,22313,U,O,NaN,Q,NaN,NaN,NaN,CHRIS MARSTEN,PO BOX 26141,"PO BOX 26141, ALEXANDRIA, VA 22313"


In [32]:
cm[cm['tres_nm'] == 'CHRIS MARSTON'].head()

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
2504,C00372201,REDWINGMCPAC,CHRIS MARSTON,PO BOX 26141,NaN,ALEXANDRIA,VA,22313,U,N,REP,Q,NaN,NONE,H2PA18143,"MARSTON, CHRIS",PO BOX 26141,"PO BOX 26141, ALEXANDRIA, VA 22313"
3092,C00410118,BACHMANN FOR CONGRESS,CHRIS MARSTON,PO BOX 26141,NaN,ALEXANDRIA,VA,22313,P,H,REP,Q,NaN,BACHMANN FOR PRESIDENT,H6MN06074,"MARSTON, CHRIS",PO BOX 26141,"PO BOX 26141, ALEXANDRIA, VA 22313"
3558,C00433524,DUNCAN D. HUNTER FOR CONGRESS,CHRIS MARSTON,PO BOX 1545,NaN,EL CAJON,CA,92022,P,H,REP,Q,NaN,HUNTER VICTORY FUND,H8CA52052,"MARSTON, CHRIS",PO BOX 1545,"PO BOX 1545, EL CAJON, CA 92022"
3723,C00441014,ROB WITTMAN FOR CONGRESS,CHRIS MARSTON,PO BOX 3770,NaN,OAKTON,VA,22124,P,H,REP,Q,NaN,WITTMAN VICTORY COMMITTEE,H8VA01147,"MARSTON, CHRIS",PO BOX 3770,"PO BOX 3770, OAKTON, VA 22124"
3919,C00449926,AMERICAN FUTURE FUND POLITICAL ACTION,CHRIS MARSTON,45 N HILL DR,STE 100,WARRENTON,VA,20186,U,W,NaN,M,NaN,NaN,NaN,"MARSTON, CHRIS",45 N HILL DR,"45 N HILL DR, WARRENTON, VA 20186"


In [33]:
new = 'CHRISTOPHER MARSTON'
dones.append(new)

these = findall(['c', 'marst'])

getall(these, new)

orig len: 15
new len: 144


In [34]:
findall(['st', ' martin'])

['ERNESTO MARTINEZ', 'STEVEN MARTIN']

In [35]:
cm[cm['tres_nm'] == 'STEVE MARTIN']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address


In [36]:
cm[cm['tres_nm'] == 'STEVEN MARTIN'].head()

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
1361,C00233353,AD ALLIANCE,STEVEN MARTIN,PO BOX 30844,NaN,BETHESDA,MD,20824,B,Q,NaN,M,M,NONE,NaN,"MARTIN, STEVEN JR.",PO BOX 30844,"PO BOX 30844, BETHESDA, MD 20824"
2507,C00372532,MICHAEL BURGESS FOR CONGRESS,STEVEN MARTIN,PO BOX 2334,NaN,DENTON,TX,76202,P,H,REP,Q,NaN,CARE AMERICA,H2TX26093,"MARTIN, STEVEN G. JR.",PO BOX 2334,"PO BOX 2334, DENTON, TX 76202"
3207,C00415208,LONE STAR LEADERSHIP PAC,STEVEN MARTIN,PO BOX 30844,NaN,BETHESDA,MD,20824,D,Q,NaN,M,NaN,MICHAEL BURGESS,NaN,"MARTIN, STEVEN G. JR.",PO BOX 30844,"PO BOX 30844, BETHESDA, MD 20824"
5181,C00493783,FRESHMAN AGRICULTURAL REPUBLICAN MEMBERS TRUST...,STEVEN MARTIN,PO BOX 30844,NaN,BETHESDA,MD,20824,J,H,NaN,Q,NaN,NONE,H0AL02087,"MARTIN, STEVEN G. JR.",PO BOX 30844,"PO BOX 30844, BETHESDA, MD 20824"
6843,C00551366,RIGHTNOW WOMEN PAC,STEVEN MARTIN,PO BOX 30844,NaN,BETHESDA,MD,20824,U,Q,NaN,Q,NaN,NONE,NaN,"MARTIN, STEVEN G. JR.",PO BOX 30844,"PO BOX 30844, BETHESDA, MD 20824"


In [37]:
dones.extend(findall(['st', ' martin']))

In [38]:
findall(' crat')

['BRADLEY CRATE', 'VIVIAN CRATER']

In [39]:
dones.extend(findall(' crat'))

In [40]:
findall('datwyler')

['THOMAS CHARLES DATWYLER', 'THOMAS DATWYLER', 'TOM DATWYLER']

In [41]:
new = 'THOMAS DATWYLER'
dones.append(new)

these = findall('datwyler')

getall(these, new)

orig len: 40
new len: 42


In [42]:
findall('zamore')

['JUDITH ZAMORE', 'JUDY ZAMORE']

In [43]:
new = 'JUDITH ZAMORE'
dones.append(new)

these = findall('zamore')

getall(these, new)

orig len: 122
new len: 123


In [44]:
findall(' pett')

['AARON PETTIGREW',
 'BRENDAN PETTYJOHN',
 'JARED SCOTT PETTINATO',
 'JAY PETTERSON']

In [45]:
dones.extend(findall(' pett'))

In [46]:
findall(['jen', 'may'])

['JENNIFER MAY']

In [47]:
dones.append('JENNIFER MAY')

In [48]:
# first jennifer for driskell
cm[cm['tres_nm'] == 'JENNIFER FAIRFIELD']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
7845,C00572859,GRETCHEN DRISKELL FOR CONGRESS,JENNIFER FAIRFIELD,PO BOX 464,NaN,SALINE,MI,48176,P,H,DEM,Q,NaN,NONE,H6MI07223,"FAIRFIELD, JENNIFER",PO BOX 464,"PO BOX 464, SALINE, MI 48176"


In [49]:
# second jennifer for driskell.  same?
cm[(cm['tres_nm'] == 'JENNIFER MAY') & (cm['cmte_st1'] == 'PO BOX 464')]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address


In [50]:
findall(' williamson')

['CRAIG WILLIAMSON',
 'DANNA LAVERN WILLIAMSON',
 'DANNA WILLIAMSON',
 'JARRELL WILLIAMSON',
 'JOSHUA WILLIAMSON',
 'LES WILLIAMSON',
 'NANCY MARIE WILLIAMSON',
 'SEAN WILLIAMSON']

In [51]:
dones.extend(findall(' williamson'))

In [52]:
findall(' ky')

['CHUCK KYRISH',
 'DICK KYRO',
 'JANICA KYRIACOPOULOS',
 'PAUL KYRILLIDIS',
 'SAMMY KYE']

In [53]:
dones.extend(findall(' ky'))

In [54]:
findall(' watkins')

['CATHLEEN WATKINS',
 'CHARLES WATKINS',
 'CHRISTOPHER CHRISTIAN WATKINS',
 'ERIK WATKINS',
 'MICHAEL WATKINS',
 'NANCY WATKINS',
 'STEVEN WATKINS',
 'WILLIAM WATKINS']

In [55]:
cm[cm['tres_nm'] == 'BILL WATKINS']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address


In [56]:
cm[cm['tres_nm'] == 'WILLIAM WATKINS']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
2551,C00376244,KUSTOFF FOR CONGRESS 2002,WILLIAM WATKINS,1661 AARON BRENNER DR,SUITE 300,MEMPHIS,TN,38120,A,H,REP,Q,NaN,NONE,H2TN07103,"WATKINS, WILLIAM H JR",1661 AARON BRENNER DRIVE,"1661 AARON BRENNER DR, MEMPHIS, TN 38120"
9537,C00607937,KELSEY FOR CONGRESS,WILLIAM WATKINS,110 E MULBERRY ST,SUITE 200,COLLIERVILLE,TN,38017,P,H,REP,Q,NaN,NONE,H6TN08151,"WATKINS, WILLIAM H. III",110 E. MULBERRY STREET,"110 E MULBERRY ST, COLLIERVILLE, TN 38017"
10036,C00614826,KUSTOFF FOR CONGRESS,WILLIAM WATKINS,1661 AARON BRENNER DR,STE 300,MEMPHIS,TN,38120,P,H,REP,Q,NaN,NaN,H2TN07103,"WATKINS, WILLIAM H JR",1661 AARON BRENNER DR,"1661 AARON BRENNER DR, MEMPHIS, TN 38120"


In [57]:
new = 'WILLIAM WATKINS'
dones.append(new)

these = ['BILL WATKINS', 'WILLIAM WATKINS']

getall(these, new)

orig len: 3
new len: 3


In [58]:
dones.extend(findall(' watkins'))

In [59]:
findall('hobbs')

['CABELL HOBBS', 'JENNIFER HOBBS', 'NICOLE HOBBS', 'SCOTT HOBBS']

In [60]:
findall('giles')

['JASON GILES', 'RODNEY GILES']

In [61]:
dones.extend(findall('giles'))

In [62]:
dones.extend(findall('hobbs'))

In [63]:
findall(' satt')

['ADAM SATTERFIELD', 'DAVID SATTERFIELD', 'STACEY SATTERLEE']

In [64]:
dones.extend(findall(' satt'))

In [65]:
findall(' curtis')

['BOBBY CURTIS BRACKETT',
 'DAVID CURTIS',
 'DAVID MARLOW CURTIS',
 'ELIZABETH CURTIS',
 'JAMES CURTIS',
 'KEVIN CURTIS',
 'LIZ CURTIS',
 'MICHAEL CURTIS DEAN',
 'RON CURTIS',
 'STEVEN DALE CURTIS']

In [66]:
findall('brackett')

['BOBBY CURTIS BRACKETT']

In [67]:
findall(['mi', 'dean'])

['CHRISTINA CAMILLE "CAMI" STRADER-DEAN', 'MICHAEL CURTIS DEAN']

In [68]:
new = 'ELIZABETH CURTIS'
dones.append(new)

these = findall(['liz', 'curtis'])

getall(these, new)

orig len: 29
new len: 32


In [69]:
dones.extend(findall(' curtis'))

In [70]:
findall(' davis')

['ADAM DAVIS',
 'BILLY DAVIS',
 'BRADLEY DAVIS',
 'BRIAN DAVIS',
 'BROOKE DAVIS',
 'CASEY DAVISON',
 'CHERISH FOREMAN DAVIS',
 'CLAY PARKER DAVIS',
 'CRYSTAL DAVIS-TAYLOR',
 'CYNTHIA DAVIS',
 'DAN DAVIS',
 'DEBORAH DAVIS',
 'DONALD DAVIS',
 'DYLAN DAVIS',
 'GARY DAVIS',
 'HAROLD DAVIS',
 'JAMES DAVIS',
 'JEAN DAVIS',
 'JEFF DAVIS',
 'JEREMY DAVIS',
 'JOE DAVIS',
 'JOHN DAVIS',
 'JONATHAN DAVIS',
 'KAYOLKA DAVIS',
 'KEITH DAVIS',
 'KELLY DAVIS',
 'KETH DAVIS',
 'LANCE DAVIS',
 'LESLIE WALTER DAVIS',
 'LUKE DAVIS',
 'MAMIE DAVIS',
 'MARIAN DAVIS',
 'MARIE ELIZABETH DAVIS',
 'MARIENELLA DAVIS',
 'MARILYN DAVIS HOLLOMAN',
 'MARK DAVIS',
 'MARK WAYNE DAVIS',
 'MICHAEL DAVIS',
 'MICHELE DAVISON',
 'MIKE DAVIS',
 'MILTON DAVIS',
 'PAUL DAVIS',
 'REGINALD DAVIS',
 'RICHARD BERNARD DAVIS',
 'RICK DAVISON',
 'ROBERT DAVIS',
 'SANDRA DAVIS',
 'SARAH DAVIS BAUER',
 'SHEILA DAVIS',
 'SUSAN DAVIS',
 'VANDY DAVIS',
 'VERA DAVIS',
 'VIRGINIA DAVIS',
 'WAYNE DAVIS',
 'WILLIAM DAVIS']

In [71]:
cm[cm['tres_nm'].apply(lambda x: x in ['BILLY DAVIS', 'WILLIAM DAVIS'])]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
9922,C00613216,THE CANDIDATE BILLY DAVIS INC.,BILLY DAVIS,1455 WIMBLEDON DR,NaN,KENNESAW,GA,30144,P,H,REP,Q,NaN,NaN,H6GA11074,"DAVIS, BILLY",1455 WIMBLEDON DRIVE,"1455 WIMBLEDON DR, KENNESAW, GA 30144"
12062,C00638536,FRIENDS OF DANNER KLINE,WILLIAM DAVIS,PO BOX 430191,NaN,VESTAVIA HILLS,AL,35243,P,H,DEM,Q,NaN,NONE,H8AL06099,"DAVIS, WILLIAM",PO BOX 430191,"PO BOX 430191, VESTAVIA HILLS, AL 35243"


In [72]:
cm[cm['tres_nm'] == 'KETH DAVIS']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
14187,C00665232,WIN IN 2018,KETH DAVIS,228 S WASHINGTON ST,SUITE 115,ALEXANDRIA,VA,22314,J,N,NaN,Q,NaN,NONE,NaN,"DAVIS, KETH A.",228 S. WASHINGTON STREET,"228 S WASHINGTON ST, ALEXANDRIA, VA 22314"


In [73]:
cm[cm['tres_nm'] == 'KEITH DAVIS'].head()

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
212,C00027466,NRSC,KEITH DAVIS,425 2ND ST NE,NaN,WASHINGTON,DC,20002,U,Y,REP,M,NaN,2018 TILLIS SENATE CANDIDATE FUND,NaN,"DAVIS, KEITH",425 2ND STREET NE,"425 2ND ST NE, WASHINGTON, DC 20002"
395,C00075820,NRCC,KEITH DAVIS,320 FIRST ST SE,.,WASHINGTON,DC,20003,U,Y,REP,M,NaN,COLE COMBINED COMMITTEE,NaN,"DAVIS, KEITH A.",320 FIRST STREET SE,"320 FIRST ST SE, WASHINGTON, DC 20003"
2013,C00330720,TRUST PAC TEAM REPUBLICANS FOR UTILIZING SENSI...,KEITH DAVIS,228 S WASHINGTON ST,SUITE 115,ALEXANDRIA,VA,22314,D,Q,NaN,Q,NaN,NONE,NaN,"DAVIS, KEITH A.",228 S. WASHINGTON STREET,"228 S WASHINGTON ST, ALEXANDRIA, VA 22314"
2659,C00383745,ALEXANDER FOR SENATE 2020 INC,KEITH DAVIS,228 S WASHINGTON ST,SUITE 115,ALEXANDRIA,VA,22314,P,S,REP,Q,NaN,NaN,S2TN00058,KEITH A DAVIS,228 S WASHINGTON STREET,"228 S WASHINGTON ST, ALEXANDRIA, VA 22314"
2738,C00388421,TENN POLITICAL ACTION COMMITTEE INC (TENN PAC),KEITH DAVIS,228 S WASHINGTON ST,NaN,ALEXANDRIA,VA,22314,D,Q,NaN,Q,NaN,NONE,NaN,"DAVIS, KEITH A.",228 S WASHINGTON STREET SUITE 115,"228 S WASHINGTON ST, ALEXANDRIA, VA 22314"


In [74]:
new = 'KEITH DAVIS' 
dones.append(new)

these = ['KEITH DAVIS', 'KETH DAVIS']

getall(these, new)

orig len: 66
new len: 67


In [75]:
cm[cm['tres_nm'].apply(lambda x: x in ['MIKE DAVIS', 'MICHAEL DAVIS'])]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
14337,C00666826,JEFF MILLER FOR NEVADA,MIKE DAVIS,3325 NORTH NELLIS,NaN,LAS VEGAS,NV,89115,P,H,REP,Q,NaN,NONE,H8NV04067,"DAVIS, MIKE",3325 NORTH NELLIS,"3325 NORTH NELLIS, LAS VEGAS, NV 89115"
15960,C00683573,STRENGTH AND PROGRESS INC,MICHAEL DAVIS,30 N GOULD ST,STE 7981,SHERIDAN,WY,82801,U,N,NaN,Q,NaN,NaN,NaN,"DAVIS, MICHAEL",30 N GOULD ST,"30 N GOULD ST, SHERIDAN, WY 82801"


In [76]:
dones.extend(findall(' davis'))

In [77]:
findall(['phillips', 'ob'])

['ROB PHILLIPS', 'ROBERT BRETON PHILLIPS', 'ROBERT PHILLIPS', 'ROBET PHILLIPS']

In [78]:
new = 'ROBERT PHILLIPS' 
dones.append(new)

these = findall(['phillips', 'ob'])

getall(these, new)

orig len: 24
new len: 31


In [79]:
findall(' mele')

['STEVE MELE', 'STEVEN MELE']

In [80]:
cm[cm['tres_nm'] == 'STEVE MELE']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
5594,C00507574,MOTOR CITY PAC,STEVE MELE,611 PENNSYLVANIA AVE SE,STE 143,WASHINGTON,DC,20003,D,Q,NaN,M,NaN,NaN,NaN,"MELE, STEVE",611 PENNSYLVANIA AVENUE SE,"611 PENNSYLVANIA AVE SE, WASHINGTON, DC 20003"
12747,C00650366,LESLIE COCKBURN FOR CONGRESS,STEVE MELE,PO BOX 186,NaN,SPERRYVILLE,VA,22740,P,H,DEM,Q,NaN,NONE,H8VA05155,"MELE, STEVE",PO BOX 186,"PO BOX 186, SPERRYVILLE, VA 22740"
14127,C00664599,ELLEN LIPTON FOR CONGRESS,STEVE MELE,PO BOX 578,NaN,ROYAL OAK,MI,48068,P,H,DEM,T,NaN,NaN,H8MI09134,"MELE, STEVE",PO BOX 578,"PO BOX 578, ROYAL OAK, MI 48068"
16182,C00685842,THE ARENA CANDIDATE PAC HOUSE VICTORY FUND,STEVE MELE,611 PENNSYLVANIA AVE SE,NUM 143,WASHINGTON,DC,20003,J,H,NaN,T,NaN,NaN,NaN,"MELE, STEVE",611 PENNSYLVANIA AVE SE,"611 PENNSYLVANIA AVE SE, WASHINGTON, DC 20003"
16851,C00692988,ALL FOR OUR COUNTRY VICTORY FUND,STEVE MELE,611 PENNSYLVANIA AVE SE,STE. 143,WASHINGTON,DC,20003,J,N,NaN,Q,NaN,NONE,NaN,"MELE, STEVE",611 PENNSYLVANIA AVENUE SE,"611 PENNSYLVANIA AVE SE, WASHINGTON, DC 20003"


In [81]:
cm[cm['tres_nm'] == 'STEVEN MELE'].head()

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
6138,C00526657,TOWARD TOMORROW PAC,STEVEN MELE,PO BOX 544,NaN,SAN ANTONIO,TX,78292,D,Q,NaN,M,NaN,NaN,NaN,"MELE, STEVEN",PO BOX 544,"PO BOX 544, SAN ANTONIO, TX 78292"
8535,C00583104,CORTEZ MASTO VICTORY FUND,STEVEN MELE,611 PENNSYLVANIA AVE SE,NaN,WASHINGTON,DC,20003,J,N,NaN,T,NaN,NaN,NaN,STEVEN MELE,611 PENNSYLVANIA AVE SE SUITE 143,"611 PENNSYLVANIA AVE SE, WASHINGTON, DC 20003"
9460,C00606939,ROSEN FOR NEVADA,STEVEN MELE,PO BOX 27195,NaN,LAS VEGAS,NV,89126,P,S,DEM,Q,NaN,ROSEN VICTORY FUND,S8NV00156,"MELE, STEVEN",PO BOX 27195,"PO BOX 27195, LAS VEGAS, NV 89126"
10915,C00626317,GREAT LAKES VICTORY FUND,STEVEN MELE,611 PENNSYLVANIA AVE SE,#143,WASHINGTON,DC,20003,J,N,NaN,Q,NaN,NONE,S4MI00355,"MELE, STEVEN",611 PENNSYLVANIA AVE. SE,"611 PENNSYLVANIA AVE SE, WASHINGTON, DC 20003"
10991,C00627232,IVOTE ACTION FUND,STEVEN MELE,722 12TH ST NW,3RD FLOOR,WASHINGTON,DC,20005,U,O,NaN,Q,NaN,NONE,NaN,"MELE, STEVEN",722 12TH ST NW,"722 12TH ST NW, WASHINGTON, DC 20005"


In [82]:
new = 'STEVEN MELE'
dones.append(new)

these = findall(['steve', ' mele'])

getall(these, new)

orig len: 35
new len: 40


In [83]:
findall(['su', 'jackson'])

['SUE JACKSON', 'SUSAN JACKSON']

In [84]:
new = 'SUSAN JACKSON'
dones.append(new)

these = findall(['su', 'jackson'])

getall(these, new)

orig len: 5
new len: 26


In [85]:
findall(' otte')

['BENJAMIN OTTENHOFF', 'TIMOTHY OTTEN']

In [86]:
new = 'BENJAMIN OTTENHOFF'
dones.append(new)

these = findall(' otte')

getall(these, new)

orig len: 56
new len: 57


In [87]:
findall(' mcmi')

['ANN MCMILLAN',
 'BENIAH MCMILLER',
 'COLLIN MCMICHAEL',
 'EVA MCMILLAN',
 'GEETA MCMILLAN',
 'JAMES MCMILLAN',
 'KURT MCMILLAN',
 'MARY MCMILLAN']

In [88]:
dones.extend(findall(' mcmi'))

In [89]:
findall(' lew')

['ANDY LEWIS JAMES',
 'CHERYL LEWIS',
 'CORDELIA LEWIS BURKS',
 'DALE LEWIS',
 'DEBORAH LEWIS',
 'DENISE LEWIS',
 'DONALD THOMAS LEWIS',
 'EMANUEL LEWIS',
 'JACK LEWIS CHARBONNEAU',
 'JEFF LEWIS',
 'JENNIFER LEWIS',
 'KAREN LEWIS',
 'KATHRYN LEWIS',
 'KEET LEWIS',
 'LARRY LEWIS',
 'LEROY LEWIS',
 'LINDSAY LEWIS',
 'MARC LEWKOWITZ',
 'MARK LEWIS',
 'MARY PARKER LEWIS',
 'MICHAEL LEWIS',
 'NANCY LEWIS',
 'NATHAN LEWIS WURTZEL',
 'PAUL LEWIS',
 'ROBERT LEWIS BUCHANAN',
 'RONNIE LEWIS ALSTON',
 'ROSS BAZELON LEWIN',
 "SABRINA Y'VES LEWIS-JONES",
 'SAMMY LEWITTES',
 'SAMUEL LEWIS GASKINS',
 'SANDRA LEWIS',
 'SCOTT LEWIS',
 'SHANTI LEWALLEN',
 'WENDY LEWIS',
 'WILLIAM LEWIS FANATIA']

In [90]:
findall(['and', ' james'])

['ALEX-ST JAMES ANDREW RAILEY-CISCO',
 'ANDREW JAMES MCDOWELL',
 'ANDY LEWIS JAMES',
 'BRANDON MICHAEL JAMES',
 "BRIAN JAMES CANDIDATE O'NEILL",
 'CHANDLER JAMES HOUGHTLEN']

In [91]:
findall(['cord', 'burk'])

['CORDELIA LEWIS BURKS']

In [92]:
findall('charbon')

['JACK LEWIS CHARBONNEAU', 'MARY CHARBONNEAU']

In [93]:
findall(['nat', 'wurtz'])

['NATHAN LEWIS WURTZEL']

In [94]:
findall(' alston')

['DONNA JEAN ALSTON', 'LINDA ALSTON', 'RONNIE LEWIS ALSTON', 'SHANTAL ALSTON']

In [95]:
cm[cm['tres_nm'] == 'WENDY LEWIS']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
3934,C00450411,MOMENTUM PAC,WENDY LEWIS,1200 SMITH,SUITE 1600,HOUSTON,TX,77002,U,N,NaN,A,NaN,NaN,NaN,WENDY LEWIS,1200 SMITH,"1200 SMITH, HOUSTON, TX 77002"


In [96]:
cm[cm['tres_nm'] == 'WENDY LEWIS ARMSTRONG']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address


In [97]:
new = 'WENDY LEWIS'
dones.append(new)

these = findall('wendy lewis')

getall(these, new)

orig len: 1
new len: 1


In [98]:
dones.extend(findall(' lew'))

In [99]:
findall(' lawl')

['CHRISTOPHER LAWLOR', 'KELLY LAWLER', 'SEWARD LAWLOR']

In [100]:
dones.extend(findall(' lawl'))

In [101]:
findall(' crum')

['GARY CRUMMITT', 'JANE CRUMPLER', 'JANET ROSE CRUM-REMBACKI', 'JOHN CRUMP']

In [102]:
dones.extend(findall(' crum'))

In [103]:
findall(' slater')

['ANNE SLATER', 'JEN SLATER', 'TODD SLATER', 'VICTORIA SLATER-MADERT']

In [104]:
dones.extend(findall(' slater'))

In [105]:
findall(' mccaul')

['ALICIA MCCAULEY', 'MICHAEL MCCAULEY', 'MIKE MCCAULEY']

In [106]:
dones.extend(findall(' mccaul'))

In [107]:
findall(' montg')

['CHRISTOPHER MONTGOMERY WOODFIN',
 'DAVA MONTGOMERY',
 'DIANE MONTGOMERY',
 'JAY MONTGOMERY',
 'LUKE MONTGOMERY',
 'MEGAN MONTGOMERY',
 'RUSSELL MONTGOMERY',
 'SHARON LYNN MONTGOMERY',
 'THOMAS MONTGOMERY',
 'TODD MONTGOMERY OIFER']

In [108]:
dones.extend(findall(' montg'))

In [109]:
findall(' nis')

[]

In [110]:
dones.extend(findall(' nis'))

In [111]:
findall(' matthews')

['LAURA MATTHEWS', 'PAUL MATTHEWS', 'RICHARD MATTHEWS']

In [112]:
dones.extend(findall(' matthews'))

In [113]:
findall(' goul')

['ANN MARIE GOULD', 'DAVID GOULD', 'JANICE GOULD', 'VAN CHARLES GOULD']

In [114]:
cm[cm['tres_nm'].apply(lambda x: ('ADAM' in x) & ('GOULD' in x))]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address


In [115]:
new = 'ADAM KOHL GOULD'
dones.append(new)

these = findall(['adam', 'gould'])

getall(these, new)

orig len: 0
new len: 0


In [116]:
dones.extend(findall(' goul'))

In [117]:
findall(' gan')

['CHARLES GANTT',
 'CHRISTOPHER JOHN GANSEN',
 'HARVEY GANTT',
 'JUSTIN GANDINO-SAADEIN',
 'MARIA VICTORIA GANACIAS BORJA',
 'MARY RANDOLPH GANNON',
 'PETER GANGI',
 'RON GANT']

In [118]:
findall('borja')

['MARIA VICTORIA GANACIAS BORJA']

In [119]:
findall(['mary', 'randolph'])

['MARY RANDOLPH GANNON']

In [120]:
dones.extend(findall(' gan'))

In [121]:
findall(' carli')

['HELEN CARLIN', 'JAMES CARLISLE', 'MICHAEL CARLIN', 'ROBERT CARLIN']

In [122]:
dones.extend(findall(' carli'))

In [123]:
findall(' kra')

['ALLEN KRAMER',
 'BETH KRATOCHVIL',
 'BLAKE ALAN KRAPF',
 'CATHERINE KRANTZ',
 'DAVID KRALLE',
 'DAVID KRAMER',
 'DEBORAH KATHRYN KRAUSE',
 'DON KRAUS',
 'JASON KRAAYVANGER',
 'JOHN KRALL',
 'PATRICK KRASON',
 'SHAWN KRAUSE',
 'STEVEN KRAVITZ',
 'TOM KRAUSE']

In [124]:
cm[cm['tres_nm'] == 'THOMAS KRAUS']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address


In [125]:
cm[cm['tres_nm'] == 'TOM KRAUSE']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
15667,C00680504,BROADCOM INC. POLITICAL ACTION COMMITTEE (BROA...,TOM KRAUSE,1320 RIDDER PARK DR,NaN,SAN JOSE,CA,95131,U,Q,NaN,M,C,NaN,NaN,"KRAUSE, TOM",1320 RIDDER PARK DR.,"1320 RIDDER PARK DR, SAN JOSE, CA 95131"


In [126]:
dones.extend(findall(' kra'))

In [127]:
findall(' koch')

['KEVIN KOCH',
 'LINDA KOCH',
 'NICHOLAS KOCH',
 'THEODORE KOCH',
 'THOMAS NEIL KOCH',
 'TIMOTHY KOCH',
 'TOM KOCHAN',
 'VICTOR LEE KOCHER']

In [128]:
dones.extend(findall(' koch'))

In [129]:
findall(' win')

['ALAN WINDGASSEN',
 'ALISON WINGATE',
 'ANDREW WINTERING',
 'ARTHUR WINSTEAD',
 'BOBBY WINT',
 'BRIGEN WINTERS',
 'DEAN WINE',
 'DONNA FOLEY WINTERSON',
 'ELIZABETH WINN BJORK',
 'ERNEST WINDHAUSER',
 'ETHAN WINGFIELD',
 'GARY WINKLER',
 'GERALD WINDHAM',
 'GRETCHEN LYNN WINNER',
 'HICKS WINTERS',
 'JOHN WINKLER',
 'JOSEPH WINTEREGG',
 'KAREN WINGARD',
 'MICHAEL WINTERHALTER',
 'PATRICIA WINTER',
 'ROBERT LEROY WINTER',
 'ROBERT VAN WINTON',
 'STEVE WIND',
 'THOMAS FRANCIS WINTERBOTTOM',
 'THOMAS WINTER',
 'TRAVIS WINES',
 'VICKIE WINPISINGER',
 'VICTORIA WINTRODE']

In [130]:
dones.extend(findall(' win'))

In [131]:
findall(' rit')

['ANDREW RITTENBERG',
 'ERIN RITTER',
 'FATIMA RITA FAHMY',
 'JOEL RITER',
 'MAX WILSON RITTER',
 'TOM RITTER']

In [132]:
dones.extend(findall(' rit'))

In [133]:
findall(' tat')

['ADAM TATUN',
 'ALIX TATE',
 'BRANDON TATE',
 'BRIAN TATUM',
 'DARRYL TATTRIE',
 'ELIZABETH TATE',
 'JOHN TATE',
 'MARTHA SHAW TATE',
 'SAMBA TATA',
 'SRINIVASA TATINENI',
 'STANLEY TATE']

In [134]:
cm[cm['tres_nm'].apply(lambda x: x in ['BRIAN TATUM', 'BRYAN TATUM'])]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
3203,C00415026,AMERICAN FUELS AND PETROCHEMICAL MANUFACTURERS...,BRIAN TATUM,1800 M ST NW,SUITE 900 NORTH,WASHINGTON,DC,20036,B,Q,NaN,M,T,AMERICAN FUELS AND PETROCHEMICAL MANUFACTURERS...,NaN,"TATUM, BRIAN",1800 M STREET NW,"1800 M ST NW, WASHINGTON, DC 20036"


In [135]:
dones.extend(findall(' tat'))

In [136]:
findall('ralls')

['STEVE RALLS', 'STEVEN RALLS']

In [137]:
cm[cm['tres_nm'] == 'STEVEN RALLS']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
1966,C00326629,KLINE FOR CONGRESS,STEVEN RALLS,332 W LEE HWY,STE 303,WARRENTON,VA,20186,U,H,REP,T,NaN,NaN,H8MN06047,"RALLS, STEVEN",332 W LEE HWY,"332 W LEE HWY, WARRENTON, VA 20186"
2845,C00395467,JEFF FORTENBERRY FOR UNITED STATES CONGRESS,STEVEN RALLS,PO BOX 30265,NaN,LINCOLN,NE,68503,P,H,REP,Q,NaN,NONE,H4NE01064,"RALLS, STEVEN",PO BOX 30265,"PO BOX 30265, LINCOLN, NE 68503"


In [138]:
cm[cm['tres_nm'] == 'STEVE RALLS'].head()

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
3389,C00426494,FOR OUR REPUBLIC'S TRADITIONS FUND AKA FORT FUND,STEVE RALLS,PO BOX 30883,NaN,LINCOLN,NE,68503,D,N,NaN,M,NaN,JEFF FORTENBERRY,NaN,"RALLS, STEVE MR.",PO BOX 30883,"PO BOX 30883, LINCOLN, NE 68503"
3416,C00427781,ONLINE LENDERS ALLIANCE POLITICAL ACTION COMMI...,STEVE RALLS,PO BOX 15480,SE STATION,WASHINGTON,DC,20003,U,Q,NaN,M,M,ONLINE LENDERS ALLIANCE,NaN,"RALLS, STEVE",PO BOX 15480,"PO BOX 15480, WASHINGTON, DC 20003"
3703,C00440115,STEVE AUSTRIA FOR CONGRESS,STEVE RALLS,2537 OBETZ DR,NaN,BEAVERCREEK,OH,45434,P,H,REP,Q,NaN,NONE,H8OH07065,"RALLS, STEVE",2537 OBETZ DR,"2537 OBETZ DR, BEAVERCREEK, OH 45434"
5012,C00489336,WILD AND WONDERFUL PAC,STEVE RALLS,332 W LEE HWY,# 303,WARRENTON,VA,20186,D,Q,NaN,M,NaN,NaN,NaN,"RALLS, STEVE",332 W LEE HWY,"332 W LEE HWY, WARRENTON, VA 20186"
5345,C00498345,MULLIN FOR CONGRESS,STEVE RALLS,PO BOX 3681,NaN,MUSKOGEE,OK,74402,P,H,REP,Q,NaN,MULLIN VICTORY FUND,H2OK02083,"RALLS, STEVE",PO BOX 3681,"PO BOX 3681, MUSKOGEE, OK 74402"


In [139]:
new = 'STEVE RALLS'
dones.append(new)

these = findall('ralls')

getall(these, new)

orig len: 28
new len: 30


In [140]:
findall(' has')

['BECKY HASSLEN',
 'CHRISSIE HASTIE',
 'DOUGLAS DONALD HASSETT',
 'GARY HASTY',
 'JESSICA HASTINGS',
 'JORDAN ALEXANDRIA TAYLOR HASTY',
 'JOSEPH HASTO',
 'KHAWAR HASSAN',
 'MARK CHARLES HASTINGS',
 'MUAD HASSAN',
 'NEIL HASSETT']

In [141]:
dones.extend(findall(' has'))

In [142]:
findall(' buch')

['EMILY BUCHANAN',
 'JOE BUCHMAN',
 'KATHERINE BUCHANAN',
 'KIMBERLY BUCHAN',
 'MARY ANNE BUCHANAN',
 'PATRICIA BUCHANAN',
 'RACHEL BUCHANAN',
 'ROBERT LEWIS BUCHANAN',
 'SHAWN BUCHTEL',
 'TIM BUCHE']

In [143]:
dones.extend(findall(' buch'))

In [144]:
findall(' bac')

['ANDREW BACON',
 'BENJAMIN BACKER',
 'CATHARYNE BACH',
 'DAN BACHUS',
 'DAN BACKER',
 'DANIEL BACINE',
 'EAT BACON',
 'JA BACHMAN',
 'JOE BACA',
 'JOEL BACON',
 'JOSEPH BACHMAN',
 'SHAWN BACHAR',
 'WILLIAM BACHMAN']

In [145]:
dones.extend(findall(' bac'))

In [146]:
findall(' brog')

['KEVIN BROGHAMER', 'LEE ANNE BROGOWSKI', 'MICHAEL BROGAN', 'RICHARD BROGAN']

In [147]:
dones.extend(findall(' brog'))

In [148]:
findall(' gia')

['ANDREW GIANNONE',
 'FRANCIS GIARDIELLO',
 'HOLLY GIARRAPUTO',
 'JACOB GIARDINA',
 'JOEL GIANNELLI',
 'LISA ANNE GIANUKOS',
 'PAMELA GENE GIAMPAPA',
 'SAVERIO GIAMBALVO',
 'SUSAN GIANNETTI LONGACRE']

In [149]:
findall('longacre')

['SUSAN GIANNETTI LONGACRE']

In [150]:
dones.extend(findall(' gia'))

In [151]:
findall(' deane')

['SHAWNDA DEANE']

In [152]:
dones.extend(findall(' deane'))

In [153]:
findall(' hag')

['DEBORAH RUTH HAGAR',
 'GLENN HAGEN',
 'HILLARY HAGERTY',
 'JANICE GWYN HAGERMAN',
 'JOHN HAGY',
 'LORA HAGGARD',
 'PAUL HAGY',
 'RICHARD HAGEN',
 'ROGER HAGAN',
 'SARAH HAGER',
 'TIMOTHY HAGAN']

In [154]:
dones.extend(findall(' hag'))

In [155]:
findall(' anger')

['LINDSAY ANGERHOLZER']

In [156]:
dones.extend(findall(' anger'))

In [157]:
findall(' lowe')

['AARON LOWE',
 'DONALD EUGENE LOWE',
 'GRETCHEN LOWE',
 'JASON LOWELL BENNETT',
 'JENNIFER LOWE',
 'KEITH LOWEY',
 'LUCAS LOWELL EASLEY',
 'MARIAN LOWERY VENABLE',
 'PATRICK LOWERY',
 'ROSS LOWE',
 'STEPHANIE KAY LOWENTHAL',
 'TRAVIS LOWE']

In [158]:
dones.extend(findall(' lowe'))

In [159]:
findall(' bau')

['BRIAN BAUER',
 'CARL BAUMAN',
 'DAVID BAUER',
 'DAWN BAUMAN',
 'GARY BAUER',
 'JUNE BAUSMAN',
 'KENNETH BAUER',
 'LINDA BAUER DARR',
 'NATALIE BAUR',
 'ORPHEAO BAUM',
 'PAULA BAUER',
 'ROBERT BAUTER',
 'SARAH DAVIS BAUER',
 'TED BAUER',
 'TIMOTHY RYAN BAUER']

In [160]:
findall(['ju', 'mart'])

[]

In [161]:
dones.extend(findall(' bau'))

In [162]:
findall(' evans')

['BRENT EVANS',
 'BRIAN EVANS',
 'DEBI EVANS',
 'DIANE EVANS',
 'DOUG EVANS',
 'GEORGE EVANS',
 'HEATHER EVANS',
 'JAMES EVANS',
 'JIM EVANS',
 'JOHN EVANS',
 'JON EVANS',
 'KIRSTIE EVANS',
 'LYLE EVANS',
 'MARTA BLEA EVANS',
 'MELISSA ANNE EVANS',
 'MERVIN EVANS',
 'MIKE EVANS',
 'NICHOLAS EVANS',
 'NICK EVANS',
 'ROBERT EVANS',
 'WALTER EVANS']

In [163]:
findall(' doyle')

['JOHN DOYLE', 'MATTHEW DOYLE', 'PETER DOYLE', 'TIM DOYLE']

In [164]:
dones.extend(findall(' doyle'))

In [165]:
dones.extend(findall(' evans'))

In [166]:
findall(' mason')

['DAVE MASON',
 'DAVID MASON',
 'DAVID MASON SISK',
 'DIRK MASON CANTRELL',
 'DYANA MASON',
 'GIB MASON',
 'JACQUELINE MASON',
 'JAMES GRAY MASON',
 'JOHN LEONARD MASON',
 'LINDA MASON',
 'MARCUS MASON',
 'MICHAEL MASON',
 'MICHAEL THOMAS MASON',
 'STACY MASON']

In [167]:
cm[cm['tres_nm'] == 'DAVE MASON'].head()

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
3021,C00406850,BLUE CROSS BLUE SHIELD OF SOUTH CAROLINA FEDER...,DAVE MASON,INTERSTATE 20 AT ALPINE RD,NaN,COLUMBIA,SC,29214,B,Q,NaN,Q,C,NaN,NaN,"MASON, DAVE MR.",INTERSTATE 20 AT ALPINE ROAD,"INTERSTATE 20 AT ALPINE RD, COLUMBIA, SC 29214"


In [168]:
cm[(cm['tres_nm'] == 'DAVID MASON') & (cm['cmte_st'] == 'SC')].head()

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address


In [169]:
findall('cantrell')

['DIRK MASON CANTRELL']

In [170]:
cm[cm['tres_nm'] == 'MICHAEL THOMAS MASON'].head()

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
11748,C00635268,ROBERT4REP,MICHAEL THOMAS MASON,PO BOX 867614,NaN,PLANO,TX,75086,P,H,IND,Q,NaN,NONE,H8TX03131,"MASON, MICHAEL THOMAS",PO BOX 867614,"PO BOX 867614, PLANO, TX 75086"


In [171]:
cm[(cm['tres_nm'] == 'MICHAEL MASON') & (cm['cmte_st'] == 'TX')].head()

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
1574,C00268904,TRINITY INDUSTRIES EMPLOYEE POLITICAL ACTION C...,MICHAEL MASON,2525 N STEMMONS FREEWAY,NaN,DALLAS,TX,75207,B,Q,NaN,M,C,TRINITY INDUSTRIES INC.,NaN,"MASON, MICHAEL J.",2525 N. STEMMONS FREEWAY,"2525 N STEMMONS FREEWAY, DALLAS, TX 75207"


In [172]:
new = 'MICHAEL MASON'
dones.append(new)

these = findall(['michael', 'mason'])

getall(these, new)

orig len: 2
new len: 3


In [173]:
findall(' hale')

['CHRISTINA HALEY',
 'DAKOTA HALE',
 'DAN HALEY',
 'DONALD BRETT HALE',
 'JAMES HALE',
 'JOHN DAVID HALE',
 'NANCY HALEY',
 'RANDY HALE',
 'SANDRA MARIE HALE',
 'TONY HALE',
 'YVONNE HALEJKO']

In [174]:
dones.extend(findall(' mason'))

In [175]:
findall(' lloyd')

['JOHN LLOYD',
 "JUNE LA'GAY LLOYD",
 'JUSTIN LLOYD MCNEEL',
 'PHILIP LLOYD',
 'SCOTT LLOYD FENSTERMAKER',
 'WILLIS LLOYD']

In [176]:
findall(' mcneel')

['JUSTIN LLOYD MCNEEL']

In [177]:
dones.extend(findall(' lloyd'))

In [178]:
findall(' johnson')

['AIDAN JOHNSON',
 'ANDREW JOHNSON',
 'ANNE-MARIE JOHNSON LEAKE',
 'AQUASIA JOHNSON',
 'ARLEN JOHNSON',
 'ARLEY JOHNSON',
 'ARNOLD JOHNSON',
 'BARRY JOHNSON',
 'BERNADETTE JOHNSON',
 'BETTY JOHNSON',
 'BRAD JOHNSON',
 'BRIAN JOHNSON',
 'BURT JOHNSON',
 'CATHERINE JOHNSON',
 'CHARLES JOHNSON',
 'CHELSEA JOHNSON',
 'CHRIS JOHNSON',
 'CLAY JOHNSON',
 'CLIFFORD JOHNSON',
 'CLIFTON JOHNSON',
 'CONSTANCE JOHNSON',
 'COOPER JOHNSON',
 'CORY JOHNSON',
 'CRAIG JOHNSON FARRAR',
 'DAVID JOHNSON',
 'DAWN MARIE JOHNSON',
 'DEBORAH ANNE JOHNSON',
 'DIANE JOHNSON',
 'DUANE JOSEPH JOHNSON',
 'EARNELLE JOHNSON',
 'EDDIE JOHNSON',
 'EMMA JOHNSON',
 'ERIC JOHNSON',
 'ERIK ARLEN JOHNSON',
 'GEORGE JOHNSON',
 'GREGORY JOHNSON',
 'JAMES JOHNSON',
 'JAN JOHNSON',
 'JESSICA JOHNSON',
 'JONATHAN ROBERT JOHNSON',
 'JOYCE JOHNSON',
 'JUDITH JOHNSON',
 'JULIE JOHNSON',
 'JUSTINE JOHNSON',
 'KAREN JOHNSON',
 'KENNETH JOHNSON',
 'KRYSTAL SHANTRELL JOHNSON',
 'LINELL JOHNSON',
 'LORRETTA JOHNSON',
 'LOUIS JOHNSON',


In [179]:
cm[cm['tres_nm'] == 'ARLEN JOHNSON']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
13611,C00659227,COMMITTEE TO ELECT KEVIN POWERS,ARLEN JOHNSON,4 RONDEN RD,NaN,MILLVILLE,MA,01529,P,H,REP,Q,NaN,NONE,H8MA02090,"JOHNSON, R. ARLEN",4 RONDEN ROAD,"4 RONDEN RD, MILLVILLE, MA 01529"


In [180]:
cm[cm['tres_nm'] == 'ERIK ARLEN JOHNSON']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
16221,C00686238,RESISTERY PAC,ERIK ARLEN JOHNSON,275 LAKE DR,NaN,KENSINGTON,CA,94708,U,O,NaN,Q,NaN,NaN,NaN,ERIK ARLEN JOHNSON,275 LAKE DRIVE,"275 LAKE DR, KENSINGTON, CA 94708"


In [181]:
cm[cm['tres_nm'] == 'CHRIS JOHNSON']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
6169,C00528166,NATIONAL ASSOCIATION FOR FIXED ANNUITIES POLIT...,CHRIS JOHNSON,1155 F ST NW,SUITE 1050,WASHINGTON,DC,20004,U,Q,NaN,Q,T,NaN,NaN,"JOHNSON, CHRIS",1155 F ST NW,"1155 F ST NW, WASHINGTON, DC 20004"


In [182]:
cm[cm['tres_nm'] == 'CHRISTOPHER LEE JOHNSON']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address


In [183]:
cm[cm['tres_nm'] == 'JIM JOHNSON']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address


In [184]:
cm[cm['tres_nm'] == 'JAMES JOHNSON']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
1378,C00236034,WERNER ENTERPRISES INC POLITICAL ACTION COMMITTEE,JAMES JOHNSON,14507 FRONTIER RD,NaN,OMAHA,NE,68138,U,Q,NaN,Q,C,WERNER ENTERPRISES INC,NaN,"JOHNSON, JAMES L.",14507 FRONTIER ROAD,"14507 FRONTIER RD, OMAHA, NE 68138"
3162,C00412569,DENTON COUNTY REPUBLICAN VICTORY FUND,JAMES JOHNSON,2921 COUNTRY CLUB RD,NaN,DENTON,TX,76202,U,X,REP,M,M,REPUBLICAN PARTY OF TEXAS,NaN,"JOHNSON, JAMES S.",2921 COUNTRY CLUB RD #102,"2921 COUNTRY CLUB RD, DENTON, TX 76202"


In [185]:
cm[cm['tres_nm'] == 'LORETTA JOHNSON']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address


In [186]:
cm[cm['tres_nm'] == 'LORRETTA JOHNSON']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
216,C00028860,"AMERICAN FEDERATION OF TEACHERS, AFL-CIO COMMI...",LORRETTA JOHNSON,555 NEW JERSEY AVE NW,NaN,WASHINGTON,DC,20001,B,Q,NaN,M,L,"AMERICAN FEDERATION OF TEACHERS, AFL-CIO",NaN,"JOHNSON, LORRETTA","555 NEW JERSEY AVENUE, NW","555 NEW JERSEY AVE NW, WASHINGTON, DC 20001"
17341,C70002472,"AMERICAN FEDERATION OF TEACHERS, AFL-CIO",LORRETTA JOHNSON,555 NEW JERSEY AVE NW,NaN,WASHINGTON,DC,20001,U,C,NaN,Q,L,NaN,NaN,"JOHNSON, LORRETTA","555 NEW JERSEY AVENUE, N.W.","555 NEW JERSEY AVE NW, WASHINGTON, DC 20001"


In [187]:
new = 'LORRETTA JOHNSON'
dones.append(new)

these = findall(['lor', 'etta', 'johnson'])

getall(these, new)

orig len: 2
new len: 2


In [188]:
findall(['mi', 'gibson'])

['MICHAEL GIBSON']

In [189]:
dones.extend(findall(' johnson'))

In [190]:
findall(' goldstein')

['MICHAEL GOLDSTEIN', 'STEPHEN GOLDSTEIN']

In [191]:
dones.extend(findall(' goldstein'))

In [192]:
findall(' marks')

['BRUCE MARKS',
 'DOUG MARKS',
 'ELENA MARKS',
 'JEFFREY MARKS',
 'LARRY MARKS',
 'MICHELLE MARKS-OSBORNE',
 'NANCY MARKS',
 'STEPHEN MARKS']

In [193]:
dones.extend(findall(' marks'))

In [194]:
findall(' grav')

['RONALD GRAVINO',
 'SCOTT GRAVES',
 'SHARRON GRAVES',
 'STEPHANIE GRAVES',
 'SURLISA GRAVES MURPHY',
 'WILLIAM GRAVES']

In [195]:
dones.extend(findall(' grav'))

In [196]:
findall(' eich')

['ALYSSA EICHMAN', 'FRED EICHEL', 'RICHARD EICHMAN']

In [197]:
dones.extend(findall(' eich'))

In [198]:
findall(' fou')

['BRIAN FOUCART',
 'HERMAN FOUSHEE',
 'KEITH FOURNIER',
 'ROGER FOUNTAIN',
 'STEVEN FOUGHT']

In [199]:
dones.extend(findall(' fou'))

In [200]:
findall(' martin')

['ANDY MARTIN',
 'ANTONIO MARTINEZ',
 'CARL MARTIN NELSON',
 'CASSANDRA MARTINEZ',
 'CHARLES MARTIN',
 'CHRIS MARTIN',
 'CLIFFORD MARTIN',
 'DANNY WAYNE MARTIN',
 'DAVID MARTIN',
 'ED MARTIN',
 'EDWARD MARTIN',
 'ERNESTO MARTINEZ',
 'FRANCES MARTIN',
 'GAYLE MARTIN',
 'GEMMA MARTIN',
 'GREGORY MARTIN WADE',
 'JAMES RAY MARTIN',
 'JANE MARTIN',
 'JANICE MARTIN',
 'JEAN MARTINEZ',
 'JENA TONICE MARTIN',
 'JENNIFER MARTIN',
 'JOANNA MARTIN-CARRINGTON',
 'JOHN MARTIN',
 'JOHN PATRICK MARTINI',
 'JONATHAN MARTIN',
 'JOSE DANIEL MARTINES',
 'JOSEPH MARTIN',
 'KARRIE MARTIN',
 'KEVIN MARTIN',
 'LARISSA MARTINEZ',
 'LAUREN JOANNE MARTINEZ',
 'LAWRENCE ARTHUR MARTIN',
 'LILIANA MARTINEZ',
 'LISA MARTINEZ',
 'LORETTA MARTIN',
 'LOUIE CRUZ MARTINEZ',
 'MARIA MARTINEZ',
 'MARIO MARTINEZ',
 'NOVEL MARTIN',
 'PAUL MARTINO',
 'PAULETTE MARIE MARTIN',
 'ROBERT MARTINEZ',
 'ROQUE MARTINEZ',
 'SAMUEL PATTERSON MARTIN',
 'SANDRA MARTINEZ',
 'SARAH MARTIN',
 'SEAN MARTIN',
 'SHARI MARTIN',
 'STEVEN MARTIN

In [201]:
findall(' salazar')

['ARNOLD SALAZAR']

In [202]:
findall('salzburg')

['MICHAEL JOSEPH SALZBURG-FELTS']

In [203]:
cm[cm['tres_nm'] == 'STEVE MARTIN']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address


In [204]:
cm[(cm['tres_nm'] == 'STEVEN MARTIN') & (cm['cmte_st'] == 'NC')]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address


In [205]:
dones.extend(findall(' martin'))

In [206]:
findall(' davidson')

['AMY DAVIDSON',
 'CARY DAVIDSON',
 'DWIGHT DAVIDSON',
 'LISA MARIE DAVIDSON',
 'NATHAN DAVIDSON',
 'REBECCA DAVIDSON',
 'ROBERT DAVIDSON']

In [207]:
findall(['paul', 'reynolds'])

['PAUL REYNOLDS']

In [208]:
cm[cm['tres_nm'] == 'PAUL REYNOLDS']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
12660,C00649459,DIANE FOR COLORADO CD3,PAUL REYNOLDS,PO BOX 771606,NaN,STEAMBOAT SPRINGS,CO,80477,P,H,DEM,Q,NaN,DIANE MITSCH BUSH VICTORY FUND,H8CO03192,"REYNOLDS, PAUL D. MR.",PO BOX 771606,"PO BOX 771606, STEAMBOAT SPRINGS, CO 80477"
16364,C00687715,DIANE MITSCH BUSH VICTORY FUND,PAUL REYNOLDS,PO BOX 771606,NaN,STEAMBOAT SPRINGS,DC,80477,J,N,NaN,Q,NaN,NONE,NaN,"REYNOLDS, PAUL D. MR.",PO BOX 771606,"PO BOX 771606, STEAMBOAT SPRINGS, DC 80477"


In [209]:
cm[cm['tres_nm'] == 'PAUL DAVIDSON REYNOLDS']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address


In [210]:
new = 'PAUL DAVIDSON REYNOLDS'
dones.append(new)

these = findall(['paul', 'reynolds'])

getall(these, new)

orig len: 0
new len: 2


In [211]:
dones.extend(findall(' davidson'))

In [212]:
findall(' bren')

['ANNE BRENSLEY',
 'CHRISTOPHER BRENNAN',
 'DORN BRENTON',
 'ERICK BRENDAN ST JOHN',
 'GREGORY BRENNICK',
 'LAUREL BRENNAN',
 'MEGAN BRENGARTH',
 'NICHOLAS BRENT MANTANONA',
 'OZY BRENNNAN',
 'OZY ROSE BRENNAN',
 'TOM BRENEMAN',
 'TUCKER BRENNAN']

In [213]:
findall(' mantan')

['NICHOLAS BRENT MANTANONA']

In [214]:
dones.extend(findall(' bren'))

In [215]:
findall(' mont')

['ADRIAN MONTAGUE',
 'CHARLES MONTALBANO',
 'CHRISTOPHER MONTGOMERY WOODFIN',
 'DACEY MONTOYA',
 'DARRIN MONTEIRO',
 'DAVA MONTGOMERY',
 'DIANE MONTGOMERY',
 'JAY MONTGOMERY',
 'KENNETH JAMES MONTALVO',
 'LUIS MONTOYA',
 'LUKE MONTGOMERY',
 'MARIA MONTANO',
 'MEGAN MONTGOMERY',
 'MICHAEL MONT',
 'PATRICIA MONTAGUE',
 'RUSSELL MONTGOMERY',
 'SHARON LYNN MONTGOMERY',
 'THOMAS MONTGOMERY',
 'TODD MONTGOMERY OIFER']

In [216]:
dones.extend(findall(' mont'))

In [217]:
findall(' hank')

['BRENDA HANKINS']

In [218]:
dones.extend(findall(' hank'))

In [219]:
findall(' lesh')

['GREGORY LESHOCK', 'MEREDITH LESHER', 'WARREN LESHNER']

In [220]:
dones.extend(findall(' lesh'))

In [221]:
findall(' bre')

['ANNE BRENSLEY',
 'BAILEY BRESSLER',
 'BEN BREWSTER',
 'BENJAMIN BREUER',
 'CATHERINE BRESLER',
 'CHARLES BREIT',
 'CHRISTOPHER BRENNAN',
 'CHRISTOPHER SCOTT BREWER',
 'DONALD BRETT HALE',
 'DORN BRENTON',
 'ERICK BRENDAN ST JOHN',
 'GREGORY BRENNICK',
 'HANNELORE BREITMEYER-JONES',
 'JAMES TROY BREWER',
 'JOANN BREIVOGEL',
 'JONATHAN BRETT RINGHAM',
 'LAUREL BRENNAN',
 'LINDA BREATHITT',
 'MARK BREBBERMAN',
 'MEGAN BRENGARTH',
 'MICHAEL BREATHES',
 'NICHOLAS BRENT MANTANONA',
 'NICOLE BREWIN',
 'OZY BRENNNAN',
 'OZY ROSE BRENNAN',
 'PATRICIA BREED',
 'PAUL BREAZEALE',
 'PETER BREYFOGLE',
 'TOM BRENEMAN',
 'TROY BREWER',
 'TUCKER BRENNAN',
 'VICTOR BREED',
 'WADE BREWER']

In [222]:
cm[cm['tres_nm'] == 'JAMES BREWER']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address


In [223]:
cm[cm['tres_nm'] == 'JAMES TROY BREWER']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
10196,C00617142,STACY RIES SNYDER FOR CONGRESS,JAMES TROY BREWER,5500 NEVIL POINTE,NaN,BRENTWOOD,TN,37027,P,H,REP,Q,NaN,NaN,H6TN05322,JAMES TROY BREWER,5500 NEVIL POINTE,"5500 NEVIL POINTE, BRENTWOOD, TN 37027"
11743,C00635219,DAVID PAC DELIVERING AMERICAN VALUES IN DC,JAMES TROY BREWER,95 WHITE BRIDGE RD,NaN,NASHVILLE,TN,37205,D,N,NaN,Q,NaN,DAVID KUSTOFF,NaN,JAMES TROY BREWER,95 WHITE BRIDGE RD SUITE 207,"95 WHITE BRIDGE RD, NASHVILLE, TN 37205"
15870,C00682633,TENNESSEANS FOR A STRONG AMERICA PAC,JAMES TROY BREWER,95 WHITE BRIDGE RD,SUITE 207,NASHVILLE,TN,37205,U,O,NaN,Q,NaN,NONE,NaN,"BREWER, JAMES TROY",95 WHITE BRIDGE RD,"95 WHITE BRIDGE RD, NASHVILLE, TN 37205"


In [224]:
new = 'JAMES BREWER'
dones.append(new)

these = findall(['james', 'brewer'])

getall(these, new)

orig len: 0
new len: 3


In [225]:
findall(' ringham')

['JONATHAN BRETT RINGHAM']

In [226]:
dones.extend(findall(' bre'))

In [227]:
findall(' sel')

['CRAIG SELLEY',
 'CRYSTAL SELLERS',
 'GLORIA SELL',
 'GREGORY SELTZER',
 'JAMES SELASE AKPO',
 'JANET SELWAY',
 'JASON SELANDER',
 'JOANNE SELENA LOPEZ',
 'JOHN SELPH',
 'JUDITH SELZER',
 'KATHY SELVAGGIO',
 'LAURA SELKEN',
 'LAURIE SELF',
 'LINDA SELL',
 'MONIKA SELMONT',
 'PAUL SELTMAN',
 'SARAH SELENE SCOTT-STEWART',
 'STEVE SELTZER']

In [228]:
findall(['ja', 'peters'])

['JAMES WALTER PETERSON']

In [229]:
findall(['sar', 'scot'])

['SARAH HARPER SCOTT', 'SARAH SELENE SCOTT-STEWART']

In [230]:
dones.extend(findall(' sel'))

In [231]:
findall(' owen')

['DAVID OWEN',
 'ELIZABETH OWEN',
 'ETHAN OWENS',
 'GAIL DENISE OWENS',
 'JOHN OWENS',
 'KYLE LEE MASTER OWENS',
 'LINDA OWENS',
 'MARK OWEN DUNBAR',
 'MICHAEL OWENS',
 'OSCAR OWENS',
 'PATRICK OWENS',
 'ROBERT OWENS',
 'RODNEY OWEN MOSIER',
 'SELENA OWENS',
 'STACY OWENS',
 'THADDEUS OWENS',
 'WARREN PATRICK OWENS',
 'WENDI OWEN',
 'WENDY ANNOR OWENS',
 'WILLIAM TIMOTHY OWEN']

In [232]:
findall(' mosier')

['HELEN MOSIER', 'JONATHAN MOSIER', 'RODNEY OWEN MOSIER']

In [233]:
dones.extend(findall(' owen'))

In [234]:
findall(' vo')

['CHRISTOPHER VOROS',
 'DATHAN VOELTER',
 'DAVID VOGT',
 'FRANCIS VOIGT',
 'FRANK PASQUALE VOLPE',
 'HARRISON WAGNER VON DWINGELO',
 'JEN GILBERT VOSS',
 'KATHY VOSBURG',
 'LORRIE VOLLAND',
 'MARK VOGEL',
 'MARTIN VOGT',
 'MICHAEL ALAN VONTZ',
 'NIKKIE VOGEL',
 'PATRICIA VODVARKA',
 'PATRICK VOSS',
 'PAUL VOGEL',
 'PIERRE VOTEAU BEAUREGARD',
 'RADOMIR VOJTECH LUZA',
 'RICHARD VOLIVA',
 'ROBERT VOLTMANN',
 'ROGER VON TING',
 'TARYN VOGEL']

In [235]:
findall(' luza')

['RADOMIR VOJTECH LUZA']

In [236]:
dones.extend(findall(' vo'))

In [237]:
findall(' bol')

['ANGELA BOLTON',
 'APRIL BOLING',
 'CHRISTINE BOLDT',
 'DANIEL BOLLNER',
 'DAVID MICHAEL BOLDON',
 'DIANE BOLAK',
 'DONALD PATRICK BOLENA',
 'ELDAR BOLSAKOV',
 'JAMES DUANE BOLIN',
 'JEFFERY BOLTON',
 'JERRY BOLES',
 'JESSE BOLIN',
 'JOHN BOLING',
 'KIMBERLY BOLIN',
 'LISA ANN BOLTE',
 'MICHAEL BOLAND',
 'MICHAEL BOLLENTIN',
 'MICHAEL BOLTON',
 'MIKE BOLAND',
 'MOLLY BOLT',
 'NATHAN BOLT',
 'PRENTISS BOLIN',
 'REGINALD BOLDING',
 'SHELIA MARIAH BOLAR',
 'SKY BOLES',
 'SUSAN BOLHAFNER',
 'TOM BOLOVINOS',
 'VINCENT BOLLON']

In [238]:
dones.extend(findall(' bol'))

In [239]:
findall(' hub')

['AARON HUBBARD',
 'BEN HUBBY',
 'CHARLES HUBERT HAUSER',
 'EDWARD HUBBARD',
 'FRANK HUBER',
 'FREDDIE HUBBARD',
 'HARRY HUBBARD',
 'JACK HUBBARD',
 'JIM HUBBARD',
 'JUDY ANN HUBBELL',
 'JULIE HUBBARD',
 'MARY ELIZABETH HUBBARD',
 'SCOTT HUBAY',
 'THERESA HUBER',
 'TSHOMBE HUBBARD']

In [240]:
findall(' hauser')

['CHARLES HUBERT HAUSER']

In [241]:
dones.extend(findall(' hub'))

In [242]:
findall(' oz')

['NINA OZLU TUNCELI', 'WILLIAM OZANUS']

In [243]:
findall(' tunc')

['NINA OZLU TUNCELI']

In [244]:
dones.extend(findall(' oz'))

In [245]:
findall(' groe')

['ERIC GROEN', 'GIDGET KAY GROENDYK', 'REBECCA GROEN']

In [246]:
dones.extend(findall(' groe'))

In [247]:
findall(' sola')

['DEREK SOLAR', 'EDEN SOLAR PATTERSON', 'KRISTIN SOLANDER']

In [248]:
findall(['ed', 'patterson'])

['EDEN PATTERSON', 'EDEN SOLAR PATTERSON']

In [249]:
dones.extend(findall(' sola'))

In [250]:
findall(' rupp')

['BEN RUPP', 'DAVID RUPPENICKER', 'JEFFREY RUPPERT', 'LORI RUPPEN']

In [251]:
dones.extend(findall(' rupp'))

In [252]:
findall(' schi')

['ADAM VICTOR SCHILLER',
 'AMY SCHILLING',
 'ANTHONY SCHIWEIER',
 'BRADY CHARLES SCHICKINGER',
 'ERIC SCHIPPERS',
 'JAMES SCHISSER',
 'JOHN SCHIESS',
 'JOHN SCHILLING',
 'JOSEPH SCHINTZ',
 'KEVIN SCHIEFFER',
 'LOUIS SCHIAZZA',
 'MANYA SCHILPEROORT',
 'MARY SCHILLING',
 'MATTHEW SCHIEBLER',
 'NICHOLAS ADAM SCHILLER',
 'SARAH SCHIMDT']

In [253]:
dones.extend(findall(' schi'))

In [254]:
findall(' guin')

['ROBERT GUINAN']

In [255]:
dones.extend(findall(' guin'))

In [256]:
findall(' stone')

['ANN STONE',
 'ERIC STONEHAM',
 'FRANCIS CRONIN STONE',
 'JANICE STONE',
 'KIEL RICHARD STONE',
 'KRISTEN STONER',
 'LISA STONE',
 "O'LENE STONE",
 'ROBIN STONE',
 'RUTH ROCHELLE STONER',
 'RYAN STONE',
 'SAMUEL STONE',
 'SARA STONE',
 'SHARON STONES',
 'SHERRI STONE',
 'THEQUIET STONE',
 'WILLIAM STONE']

In [257]:
dones.extend(findall(' stone'))

In [258]:
findall(' cros')

['CALEB CROSBY',
 'COLLEEN CROSSEY',
 'GRADY CROSBY',
 'JAMES CROSS',
 'KAREN CROSS',
 'KENNETH CROSS',
 'MARK CROSS']

In [259]:
dones.extend(findall(' cros'))

In [260]:
findall(' gla')

['ALAN GLAZIER',
 'DANIEL GLASER',
 'EUSTACE GLASGOW',
 'EZRA GLASER',
 'JAMES GLASGOW',
 'JENNIFER GLAIM',
 'JOHN GLANCEY',
 'MITCH GLAZIER',
 'NATHANIEL GLASGOW',
 'STEVE GLADSTONE',
 'SUSAN GLAD-ANDERSON',
 'WANDA GLAUD',
 'WILLIAM GLASS']

In [261]:
findall('hemmig')

[]

In [262]:
dones.extend(findall(' gla'))

In [263]:
findall(' ragan')

['ASHLEY RAGAN', 'JACKI RAGAN', 'JENNIFER RAGAN', 'VIRGINIA RAGAN']

In [264]:
dones.extend(findall(' ragan'))

In [265]:
findall(' mau')

['BOB MAURER',
 'DAVID MAURO',
 'GEORGE MAURER',
 'LESTER MAUL',
 'LESTER SKIP MAUL',
 'STACEY MAUD',
 'TERRY MAUPIN',
 'WENDY MAUSOLF']

In [266]:
findall('littlejohn')

[]

In [267]:
findall('buchser')

[]

In [268]:
dones.extend(findall(' mau'))

In [269]:
findall(' snyd')

['CRAIG SNYDER',
 'FRANK SNYDER',
 'GARY SNYDER',
 'JASON SNYDER',
 'MICHAEL SNYDER',
 'RICHARD SNYDER',
 'SPENCER ERIC SNYDER',
 'TERRANCE SNYDER']

In [270]:
dones.extend(findall(' snyd'))

In [271]:
findall(' pur')

['BYRON PURCELL',
 'CRAIG PURSER',
 'DANEL PURKEY',
 'KENNETH PURNELL YANCY',
 'MICHAEL PURZYCKI',
 'PAULA PURDY',
 'SALVATORE PURPURA',
 'SANJAY PURI',
 'SHERRY PURDY',
 'STARR PURDUE']

In [272]:
findall(' balc')

['MARY ELLEN BALCHUNIS']

In [273]:
dones.extend(findall(' pur'))

In [274]:
findall(' lei')

['BRYAN LEIB',
 'CODY PAUL LEISTIKOW',
 'CYNTHIA LEIGH APPLEBAUM',
 'DEAN LEIGH FOWLER',
 'ERIC LEIGH KELLER',
 'GARY LEIGH',
 'GLEN LEIBOWITZ',
 'JANE LEIDERMAN',
 'JASON LEINEWEBER',
 'JEFFREY LEITER',
 'JUSTIN LEIGH FARBER',
 'KRISTI LEIGH CAPONE',
 'MIGUEL LEIJA',
 'SUSAN LEIVAS-STURNER']

In [275]:
findall('bronson')

[]

In [276]:
findall('kasz')

[]

In [277]:
findall(['cynth', 'apple'])

['CYNTHIA LEIGH APPLEBAUM']

In [278]:
findall('fowler')

['CYNTHIA FOWLER',
 'DEAN LEIGH FOWLER',
 'DONALD FOWLER',
 'JAY FOWLER',
 'JERLDEAN FOWLER',
 'PAUL FOWLER',
 'RICHARD ANTHONY FOWLER',
 'SCOTT FOWLER',
 'SEAN FOWLER',
 'STEVEN FOWLER',
 'WILMA GEORGINA FOWLER MCVAY']

In [279]:
findall('keller')

['AARON KELLER',
 'CRAIG KELLER',
 'ERIC KELLER',
 'ERIC LEIGH KELLER',
 'JAMES KELLER',
 'LYNNE KELLER',
 'SCOTT KELLER']

In [280]:
cm[cm['tres_nm'] == 'ERIC KELLER']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
16029,C00684282,EDEMOCRATS PAC LLC,ERIC KELLER,70 LINDEN AVE,P O BOX 2134,HADDONFIELD,NJ,08033,U,N,NaN,Q,NaN,NONE,NaN,"KELLER, ERIC MR",70 LINDEN AVENUE,"70 LINDEN AVE, HADDONFIELD, NJ 08033"


In [281]:
cm[cm['tres_nm'] == 'ERIC LEIGH KELLER']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
14311,C00666537,PRESS FREEDOM POLITICAL ACTION COMMITTEE,ERIC LEIGH KELLER,902 TURKEY RUN RD,NaN,MCLEAN,VA,22101,U,N,NaN,Q,NaN,NONE,NaN,"KELLER, ERIC LEIGH MR.",902 TURKEY RUN ROAD,"902 TURKEY RUN RD, MCLEAN, VA 22101"


In [282]:
findall(' hubbard')

['AARON HUBBARD',
 'EDWARD HUBBARD',
 'FREDDIE HUBBARD',
 'HARRY HUBBARD',
 'JACK HUBBARD',
 'JIM HUBBARD',
 'JULIE HUBBARD',
 'MARY ELIZABETH HUBBARD',
 'TSHOMBE HUBBARD']

In [283]:
dones.extend(findall(' lei'))

In [284]:
findall(' tea')

['DONALD TEAL', 'GARY TEAL', 'MATT TEAGARDEN', 'RICHARD TEAMAN']

In [285]:
dones.extend(findall(' tea'))

In [286]:
findall(' rut')

['CARLIN RUTH BROWN',
 'DEBORAH RUTH HAGAR',
 'DELOIS RUTH BAILEY',
 'JANNA RUTLAND',
 'JEFFREY RUTAN',
 'JOHN RUTKAUSKAS',
 'LARRY RUTH',
 'LUCY RUTISHAUSER',
 'REID RUTHERFORD',
 'RUSSELL RUTHERFORD',
 'STEPHANIE RUTH MAYO',
 'SUSAN RUTT',
 'TERRI RUTH LINNELL']

In [287]:
findall('bellar')

[]

In [288]:
findall(' mayo')

['FREDERICK MAYOCK', 'GREG ADAM MAYON', 'HEATH MAYO', 'STEPHANIE RUTH MAYO']

In [289]:
dones.extend(findall(' mayo'))

In [290]:
dones.extend(findall(' rut'))

In [291]:
findall(' elli')

['ALLEN ELLISON',
 'AMY ELLIS',
 'BRIAN JAMES ELLISON',
 'CLAIRE ELISABETH ELLIOTT',
 'GORDON ELLIOTT',
 'GREG ELLIOTT',
 'HEIDI ELLIS ROBEY',
 'JEROME ELLIOTT TRES TRUSTEE FIELDS',
 'JOE ELLISON',
 'JOHN CLELLAND ELLIS',
 'LISA ELLIS',
 'MAE NAN ELLINGSON',
 'MICHAEL ELLIOTT',
 'PAUL ELLIOTT DAHL',
 'SCOTT ELLINGTON',
 'THERESA ELLINGTON',
 'TWANA ELLIOTT']

In [292]:
findall(' robey')

['HEIDI ELLIS ROBEY']

In [293]:
findall(' dahl')

['DENNIS DAHL',
 'KATHERINE DAHLKE',
 'LISA DAHLING',
 'PAUL ELLIOTT DAHL',
 'THOMAS DAHLEN']

In [294]:
dones.extend(findall(' elli'))

In [295]:
findall(' kee')

['BRONWYN KEENAN',
 'DARRYL KEENAN SEGARS',
 'DAVID KEEN',
 'HELENE KEELEY',
 'JOHN KEELING',
 'JONATHAN DAVID KEELER',
 'LYNETTE KEETON',
 'MELISSA KEELING',
 'MICHAEL KEEGAN',
 'MICHAEL KEELING',
 'THORNTON KEEL']

In [296]:
findall(' segars')

['DARRYL KEENAN SEGARS']

In [297]:
dones.extend(findall(' kee'))

In [298]:
findall(' jan')

['AUSTIN BENNET TR JANSSEN',
 'BEVE JANE THAKHAMHOR',
 'DEBORAH JANSEN',
 'DNP APRN JANICE JONES',
 'ELAINE JANUS',
 'GORAN JANKOVIC',
 'JOHN JANKOWSKI',
 'LATREASHA JANET GIVENS',
 'LUCI JANE STUBBLEFIELD-SWEENEY',
 'MARY JANE PAVILACK',
 'MAUREEN JANES',
 'MICHELLE JANEEN WHITE',
 'REBECCA JANE BARTON',
 'SALLY JANE DAUGHERTY',
 'TIMOTHY JANISSE']

In [299]:
findall(' janssen')

['AUSTIN BENNET TR JANSSEN']

In [300]:
findall(' thakh')

['BEVE JANE THAKHAMHOR']

In [301]:
findall('osgerby')

[]

In [302]:
findall('wharton')

['CYNTHIA WHARTON', 'GEORGE WHARTON', 'GWENDOLYN WHARTON', 'KOTA WHARTON']

In [303]:
findall(' tanis')

[]

In [304]:
dones.extend(findall(' jan'))

In [305]:
findall(' dupr')

['ABBY DUPREE', 'MARIE DUPREE', 'SUSAN DUPREY']

In [306]:
dones.extend(findall(' dupr'))

In [307]:
findall(' schwartz')

['ANDREW SCHWARTZ',
 'ARTHUR SCHWARTZ',
 'DEREK SCHWARTZ',
 'JAMES SCHWARTZ',
 'JENNIFER SCHWARTZ',
 'JOHN SCHWARTZ',
 'KARL SCHWARTZ',
 'LAURA SCHWARTZ',
 'LEONARD SCHWARTZ',
 'MELODIE SCHWARTZ',
 'STEVEN SCHWARTZ']

In [308]:
cm[cm['tres_nm'] == 'LAURA SCHWARTZ']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
8348,C00580415,FASO FOR CONGRESS,LAURA SCHWARTZ,PO BOX 448,NaN,KINDERHOOK,NY,12106,P,H,REP,Q,NaN,FRESHMAN AGRICULTURAL REPUBLICAN MEMBERS TRUST...,H6NY19169,"SCHWARTZ, LAURA A. MS.",PO BOX 448,"PO BOX 448, KINDERHOOK, NY 12106"
9348,C00602755,LANGE FOR CONGRESS,LAURA SCHWARTZ,PO BOX 481,NaN,SOUTH SALEM,NY,10590,P,H,REP,Q,NaN,NONE,NaN,"SCHWARTZ, LAURA A.",PO BOX 481,"PO BOX 481, SOUTH SALEM, NY 10590"
10580,C00622100,FASO VICTORY COMMITTEE,LAURA SCHWARTZ,PO BOX 98,NaN,SOUTH SALEM,NY,10590,J,N,NaN,T,NaN,NaN,NaN,"SCHWARTZ, LAURA",PO BOX 98,"PO BOX 98, SOUTH SALEM, NY 10590"
11309,C00630681,EMPIRE STATE PAC,LAURA SCHWARTZ,PO BOX 98,NaN,SOUTH SALEM,NY,10590,D,N,NaN,Q,NaN,NaN,NaN,"SCHWARTZ, LAURA A.",PO BOX 98,"PO BOX 98, SOUTH SALEM, NY 10590"
11556,C00633263,JOHN FASO VICTORY COMMITTEE,LAURA SCHWARTZ,PO BOX 98,NaN,SOUTH SALEM,NY,10590,J,N,NaN,Q,NaN,NONE,NaN,"SCHWARTZ, LAURA",PO BOX 98,"PO BOX 98, SOUTH SALEM, NY 10590"
14735,C00670885,NEW YORK MAJORITY VICTORY,LAURA SCHWARTZ,PO BOX 98,NaN,SOUTH SALEM,NY,10590,J,H,NaN,T,NaN,NaN,NaN,"SCHWARTZ, LAURA",PO BOX 98,"PO BOX 98, SOUTH SALEM, NY 10590"
15542,C00679225,KEEP THE HOUSE,LAURA SCHWARTZ,PO BOX 98,NaN,SOUTH SALEM,NY,10590,J,N,NaN,T,NaN,NaN,NaN,LAURA SCHWARTZ,P.O. BOX 98,"PO BOX 98, SOUTH SALEM, NY 10590"


In [309]:
cm[cm['tres_nm'] == 'LAURA ANN SCHWARTZ']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address


In [310]:
dones.extend(findall(' schwartz'))

In [311]:
findall(' kau')

['AARON KAUFMAN',
 'AMIT KAUL',
 'ANNE MARIE KAUFMANN',
 'EMILY KAUFMAN',
 'JASON KAUNE',
 'JOHN KAUTTER',
 'KARL KAUFMANN',
 'LAWRENCE ROBERT OMEGA ALPHA KAUFHOLD',
 'MATTHEW KAULANA ING',
 'PHIL KAUFMAN',
 'ROBERT KAUFMANN',
 'SCOTT KAUFMAN',
 'STEPHEN KAUFMAN']

In [312]:
cm[cm['tres_nm'] == 'RON KAUFMAN']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address


In [313]:
cm[cm['tres_nm'] == 'RONALD KAUFMAN']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address


In [314]:
new = 'RONALD KAUFMAN'
dones.append(new)

these = findall(['ron', ' kaufman'])

getall(these, new)

orig len: 0
new len: 1


In [315]:
dones.extend(findall(' kau'))

In [316]:
findall(' riesc')

['JOSE RIESCO']

In [317]:
dones.extend(findall(' riesc'))

In [318]:
findall(' cop')

['ARTHUR COPLESTON',
 'BILLY COPELAND',
 'ELIZABETH COPPINGER',
 'JACKIE COPELAND',
 'KENNETH CLIFFORD MRF COPE',
 'MICHAEL COPELEY',
 'RITA COPELAND',
 'SERGIO COPPOLA',
 'VONA COPP']

In [319]:
dones.extend(findall(' cop'))

In [320]:
findall(' bal')

['AJ BALUKOFF',
 'BRIANA BALESKIE',
 'BRITTANY BALLARD',
 'JAMES BALL',
 'JENNIFER BALL',
 'JOHN BALLARD',
 'JONATHAN BALDWIN',
 'JONATHAN BALL',
 'JOSHUA BALLARD',
 'KAREN BALLARD',
 'KEITH BALKCOM',
 'KRYSTAL BALL',
 'LAURA BALDI',
 'MARY ELLEN BALCHUNIS',
 'MATANGI BALA',
 'MATTHEW BALAZIK',
 'MATTHEW BALDWIN KNIGHTS',
 'MICHAEL BALL',
 'PAUL BALASSA',
 'RICHARD BALL',
 'ROBERT BALGENORTH',
 'SAL BALDENEGRO',
 'SUSAN BALKENBUSH',
 'THOMAS BALLOCK',
 'TRACEY BALL']

In [321]:
dones.extend(findall(' bal'))

In [322]:
findall(' tall')

['ROBERT TALLIA', 'VINCENT TALLMAN']

In [323]:
dones.extend(findall(' tall'))

In [324]:
findall(' juk')

['JOEL JUKUS']

In [325]:
dones.extend(findall(' juk'))

In [326]:
findall(' milln')

[]

In [327]:
dones.extend(findall(' milln'))

In [328]:
findall(' goode')

['KATHLEEN GOODE', 'KIMBERLY GOODEN', 'MICHAEL GOODE', 'WARREN GOODE']

In [329]:
dones.extend(findall(' goode'))

In [330]:
findall(' kun')

['ADEN KUN',
 'CRAIG KUNKLE',
 'DAVE KUNES',
 'HOORIA KUNDI',
 'JOHN KUNITZ',
 'JOHN WILLIAM KUNITZ',
 'LORNA KUNEY',
 'ROBERT KUNIEGEL',
 'TEKIN KUNT',
 'THOMAS KUNA']

In [331]:
cm[cm['tres_nm'].apply(lambda x: ('KUNITZ' in x) & ('JOHN' in x))]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
10896,C00626093,THIRD CONGRESSIONAL DISTRICT REPUBLICAN PARTY ...,JOHN KUNITZ,PO 390722,NaN,EDINA,MN,55439,U,X,REP,Q,NaN,NONE,NaN,"KUNITZ, JOHN W MR.",PO 390722,"PO 390722, EDINA, MN 55439"
15499,C00678789,CONSERVATIVES FOR AMERICA,JOHN WILLIAM KUNITZ,6441 BRETTON WAY,NaN,CHANHASSEN,MN,55317,U,X,REP,Q,NaN,NONE,NaN,"KUNITZ, JOHN WILLIAM MR",6441 BRETTON WAY,"6441 BRETTON WAY, CHANHASSEN, MN 55317"


In [332]:
new = 'JOHN WILLIAM KUNITZ'
dones.append(new)

these = findall(['kunitz', 'john'])

getall(these, new)

orig len: 1
new len: 2


In [333]:
dones.extend(findall(' kun'))

In [334]:
findall('lefko')

[]

In [335]:
cm[cm['tres_nm'].apply(lambda x: 'LEFKOWITZ' in x)]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address


In [336]:
new = 'JON LEFKOWITZ'
dones.append(new)

these = findall('lefkowitz')

getall(these, new)

orig len: 0
new len: 0


In [337]:
findall(' sif')

['TIM SIFERT', 'TIMOTHY SIFERT']

In [338]:
dones.extend(findall(' sif'))

In [339]:
findall(' vand')

['CRAIG VANDERVEER',
 'DAWNE VANDIVER',
 'DICK VANDER WOUDE',
 'DIRK VANDONGEN',
 'ELIZABETH ANN VANDERWAY',
 'JASON VANDEVER',
 'MARK ALLEN VANDEWEGE',
 'MIKE VANDERWEY',
 'NIGEL VANDERFORD',
 'RAY VANDRIESSCHE',
 'SHAWN VANDIVER',
 'WILLIAM VANDERBROOK']

In [340]:
dones.extend(findall(' vand'))

In [341]:
findall(' changkyu')

[]

In [342]:
new = 'JASON CHANGKYU KIM'
dones.append(new)

these = findall(' changkyu')

getall(these, new)

orig len: 0
new len: 0


In [343]:
findall(['ash', 'newman'])

['ASHLEY NEWMAN']

In [344]:
dones.extend(findall(['ash', 'newman']))

In [345]:
findall(' rogers')

['BRETT ROGERS',
 'CHERI ROGERS',
 'CINNAMON ROGERS',
 'DENNIS ROGERS',
 'EMORY ROGERS',
 'FRANK ROGERS',
 'GEORGE ROGERS',
 'GINGER GREENE ROGERS',
 'GLEN ROGERS',
 'GRACE ROGERS',
 'JOHN ROGERS',
 'KEVIN ROGERS',
 'MARCIA MATHISON ROGERS',
 'MICHAEL ROGERS',
 'STEVEN ROGERS',
 'TERRANCE ROGERS']

In [346]:
dones.extend(findall(' rogers'))

In [347]:
findall(' kennedy')

['BONNIE KENNEDY',
 'CASTLEN KENNEDY',
 'CHRIS KENNEDY',
 'CHRISTOPHER KENNEDY',
 'GORDON KENNEDY',
 'JEANNE KENNEDY',
 'JOHN FITZGERALD KENNEDY LANKSTER',
 'KATHERINE KENNEDY',
 'KATIE KENNEDY',
 'KEVIN KENNEDY',
 'MAURA KENNEDY',
 'MICHAEL KENNEDY',
 'PATRICK KENNEDY',
 'RICHARD KENNEDY',
 'ROBERT KENNEDY',
 'SEAN KENNEDY',
 'WARD KENNEDY',
 'YOLANDA KENNEDY']

In [348]:
cm[cm['tres_nm'].apply(lambda x: x in ['CHRIS KENNEDY', 'CHRISTOPHER KENNEDY'])]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
1309,C00222455,CITIZENS FOR JOE KENNEDY 1988,CHRISTOPHER KENNEDY,C/O S KIDDER HEMENWAY & BARNES,75 STATE STREET,BOSTON,MA,02109,P,H,DEM,Q,NaN,NaN,H6MA08141,"KENNEDY, CHRISTOPHER","C/O S. KIDDER, HEMENWAY & BARNES","C/O S KIDDER HEMENWAY & BARNES, BOSTON, MA 02109"
13258,C00655621,CHRIS KENNEDY FOR COLORADO,CHRIS KENNEDY,2514 FILMORE AVE,NaN,GRAND JUNCTION,CO,81505,P,H,DEM,Q,NaN,NONE,H8CO03200,"KENNEDY, CHRIS",2514 FILMORE AVE,"2514 FILMORE AVE, GRAND JUNCTION, CO 81505"


In [349]:
cm[cm['tres_nm'].apply(lambda x: x in ['KATIE KENNEDY', 'KATHERINE KENNEDY'])]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
6627,C00546325,TAKING AN INDEPENDENT PERSPECTIVE TOGETHER FOR...,KATHERINE KENNEDY,2318 CURTIS ST,NaN,DENVER,CO,80205,D,N,NaN,Q,NaN,NaN,NaN,KATHERINE KENNEDY,2318 CURTIS STREET,"2318 CURTIS ST, DENVER, CO 80205"
7169,C00558759,PDC ENERGY INC PAC,KATHERINE KENNEDY,1775 SHERMAN ST,NaN,DENVER,CO,80203,U,Q,NaN,Q,C,NaN,NaN,"KENNEDY, KATHERINE",1775 SHERMAN STREET SUITE 3000,"1775 SHERMAN ST, DENVER, CO 80203"
7399,C00564435,PRINCIPLED CONSERVATIVES OF COLORADO,KATIE KENNEDY,600 17TH ST,NaN,DENVER,CO,80202,U,O,NaN,Q,NaN,NaN,NaN,KATIE KENNEDY,600 17TH STREET SUITE 2800 SOUTH,"600 17TH ST, DENVER, CO 80202"
8382,C00580894,VAIL RESORTS EMPLOYEE POLITICAL ACTION COMMITTEE,KATIE KENNEDY,390 INTERLOCKEN CRESCENT,NaN,BROOMFIELD,CO,80021,U,Q,NaN,Q,C,NaN,NaN,"KENNEDY, KATIE",390 INTERLOCKEN CRESCENT,"390 INTERLOCKEN CRESCENT, BROOMFIELD, CO 80021"
14698,C00670505,COMMITTEE FOR BARRINGTON,KATIE KENNEDY,745 S MILLER ST,NaN,LAKEWOOD,CO,80226,P,H,REP,Q,NaN,NONE,H8CO07094,"KENNEDY, KATIE",745 S. MILLER STREET,"745 S MILLER ST, LAKEWOOD, CO 80226"
15648,C00680306,BARRIS FOR CONGRESS,KATIE KENNEDY,24853 CHRIS DR,NaN,EVERGREEN,CO,80439,P,H,LIB,Q,NaN,NONE,H8CO02269,"KENNEDY, KATIE",24853 CHRIS DRIVE,"24853 CHRIS DR, EVERGREEN, CO 80439"
15864,C00682559,PROTECTING OUR CONSTITUTION,KATIE KENNEDY,5910 S UNIVERSITY BLVD C18 NO 254,NaN,GREENWOOD VILLAGE,CO,80121,U,O,NaN,Q,NaN,NONE,NaN,"KENNEDY, KATIE",5910 S. UNIVERSITY BLVD C18 NO 254,"5910 S UNIVERSITY BLVD C18 NO 254, GREENWOOD V..."


In [350]:
new = 'KATHERINE KENNEDY'
dones.append(new)

these = findall(['kat', 'kennedy'])

getall(these, new)

orig len: 2
new len: 7


In [351]:
cm[cm['tres_nm'].apply(lambda x: x in [
    'MICHAEL KENNEDY', 
    'MICHAEL WARREN KENNEDY', 
])]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
4648,C00477299,"VMWARE, INC. POLITICAL ACTION COMMITTEE (VMWAR...",MICHAEL KENNEDY,3401 HILLVIEW AVE,NaN,PALO ALTO,CA,94304,U,Q,NaN,M,C,NaN,NaN,"KENNEDY, MICHAEL",3401 HILLVIEW AVE,"3401 HILLVIEW AVE, PALO ALTO, CA 94304"


In [352]:
dones.extend(findall(' kennedy'))

In [353]:
findall(['mel', 'allen'])

['KYLE ALLEN HIMMELWRIGHT', 'MELISSA ALLEN']

In [354]:
findall(' himmel')

['KYLE ALLEN HIMMELWRIGHT', 'MARC HIMMELSTEIN']

In [355]:
dones.extend(findall(' himmel'))
dones.extend(findall(['mel', 'allen']))

In [356]:
findall(' reis')

['ALAN REISCHE',
 'ALEXANDER REISH',
 'BRANDON REISER',
 'DANIEL REISTETER',
 'JOHN EMANUEL REISS',
 'MICHELE REISNER',
 'SAMUEL REISBORD']

In [357]:
dones.extend(findall(' reis'))

In [358]:
findall(' doz')

['JAMES DOZIER', 'JOSEPH DOZIER', 'JULIE DOZIER']

In [359]:
dones.extend(findall(' doz'))

In [360]:
findall(' roz')

['ADAM ROZANSKY', 'COSTANTINO ROZZO', 'YURY ROZEL']

In [361]:
dones.extend(findall(' roz'))

In [362]:
findall(' patt')

['ALYX PATTISON',
 'ANDREW PATTERSON',
 'CHRIS PATTON',
 'EDEN PATTERSON',
 'EDEN SOLAR PATTERSON',
 'JAMES PATTON',
 'MELVIN PATTERSON',
 'ROBERT PATTISON',
 'SAMUEL PATTERSON MARTIN',
 'SCOTT PATTON',
 'STEPHEN PATTERSON',
 'THERESA PATTARA',
 'TOM PATTERSON',
 'WILLIAM RALPH PATTERSON']

In [363]:
dones.extend(findall(' patt'))

The work is never done...

In [364]:
# checks which have already been deduped and does not display those
mask = [False if cm.loc[i, 'tres_nm'] in dones else True for i in list(cm.index)]
cm.loc[mask, 'tres_nm'].value_counts()[cm.loc[mask, 'tres_nm'].value_counts() > 1][:20]

SCOTT MACKENZIE       16
DIANA SAQUELLA        12
BAILEY MORGAN         12
KATE LIND             12
TRAVIS KABRICK        11
SLOANE SKINNER        10
ALEXANDER HORNADAY    10
EZEKIEL FREDERICK      9
THOMAS MAXWELL         9
RUSSELL MILLER         9
CHRISTINA SIROIS       9
MEGAN MIELNIK          9
REBECCA LUBY           8
THOMAS HILTACHK        8
NANCY HALEY            8
MATTHEW YUSKEWICH      7
DEWON HUGGINS          7
RYAN PHILLIPS          7
PAUL TARNAWSKI         7
GIANNI DONATES         7
Name: tres_nm, dtype: int64

In [365]:
dupename = list(cm['cmte_nm'].value_counts()[cm['cmte_nm'].value_counts() > 1].index)

for dupe in dupename:
    lil_cm = cm.loc[cm['cmte_nm'] == dupe, [
#         'cmte_id',
        'tres_nm',
        'address',
    ]].sort_values(['address'])
    print(dupe)
    print(lil_cm)
    print('\n')

(cmte_nm missing)
                     tres_nm                                  address
8082        MARGIE NELL PENN  3401 WILCOX BLVD, CHATTANOOGA, TN 37411
16999  (no treasurer listed)       PO BOX 33524, WASHINGTON, DC 20033
8006   (no treasurer listed)                        nan, nan, nan nan
10164  (no treasurer listed)                        nan, nan, nan nan
10419  (no treasurer listed)                        nan, nan, nan nan
10519  (no treasurer listed)                        nan, nan, nan nan
12322  (no treasurer listed)                        nan, nan, nan nan
13088  (no treasurer listed)                        nan, nan, nan nan
13089  (no treasurer listed)                        nan, nan, nan nan
13396  (no treasurer listed)                        nan, nan, nan nan


COLLINS FOR CONGRESS
                   tres_nm                                 address
5967       CYNTHIA GIFFORD  176 SYCAMORE LANE, WAKEFIELD, RI 02879
9766    CATHERINE ANN PAGE  3045 DARIEN PARK DR, ROSWEL

CLINICIANS FOR PROGRESSIVE CARE
              tres_nm                                address
11603  ZACKARY BERGER  822 GUILFORD AVE, BALTIMORE, MD 21202
11623  ZACHARY BERGER  822 GUILFORD AVE, BALTIMORE, MD 21202


AMERICANS FOR COMMON SENSE
                tres_nm                                  address
8650     NICOLE CATRINA        1807 KINSELL DR, WEISER, ID 83672
13242  JOSHUA STRECKERT  2128 W RICE ST APT 2, CHICAGO, IL 60622


NEBRASKA STATE EDUCATION ASSOCIATION
                  tres_nm                           address
17423  CRAIG CHRISTIANSEN  605 S 14TH ST, LINCOLN, NE 68508
17632     SHERI JABLONSKI  605 S 14TH ST, LINCOLN, NE 68508


LEAD THE FIGHT PAC
           tres_nm                                    address
13945  LISA LISKER  228 S WASHINGTON ST, ALEXANDRIA, VA 22314
15856  LISA LISKER  228 S WASHINGTON ST, ALEXANDRIA, VA 22314


FORWARD MONTANA
                     tres_nm                              address
16570             MATT KELLY  1535 WYOMING ST, MISS

AMERICAN UPRISING PAC
                   tres_nm                                 address
10591    CHRISTIAN RICKERS  1390 CHAIN BRIDGE RD, MCLEAN, VA 22101
12210  CHRISTOPHER MARSTON      PO BOX 26141, ALEXANDRIA, VA 22313


FRIENDS OF THE EARTH (ACTION) INC
                     tres_nm                                         address
17661           MADELYN RYGG  1101 15TH ST NW 11TH FLR, WASHINGTON, DC 20005
18985  (no treasurer listed)           1101 15TH ST NW, WASHINGTON, DC 20005


SERVICE EMPLOYEES INTERNATIONAL UNION (SEIU)
                     tres_nm                                          address
16984          LIZ GUSTAFSON  1800 MASSACHUSETTS AVE NW, WASHINGTON, DC 20036
18386  (no treasurer listed)  1800 MASSACHUSETTS AVE NW, WASHINGTON, DC 20036


FRELINGHUYSEN FOR CONGRESS
             tres_nm                               address
861     ALBERT OLSEN  19 CATTANO AVE, MORRISTOWN, NJ 07960
1765  WARREN LESHNER  19 CATTANO AVE, MORRISTOWN, NJ 07960


MAJORITY FORWARD
    

SPECIAL CORPORATION FOR ETHICS AND JUSTICE IN AMERICA
                     tres_nm  \
6091     JOHN DEREK GINSBERG   
18487  (no treasurer listed)   

                                                 address  
6091   33 VILLAGE PARKWAY UNIT 122, CIRCLE PINES, MN ...  
18487    C/O JOHN DEREK GINSBERG, CIRCLE PINES, MN 55014  


COMMON SENSE OHIO
                     tres_nm                              address
18139        LANCE MCALINDON   1625 VINE ST, CINCINNATI, OH 45202
16953  (no treasurer listed)  401 9TH ST NW, WASHINGTON, DC 20004


AMALGAMATED TRANSIT UNION
                     tres_nm  \
17203        LAWRENCE HANLEY   
17146  (no treasurer listed)   

                                                 address  
17203   10000 NEW HAMPSHIRE AVE, SILVER SPRING, MD 20903  
17146  10000 NEW HAMPSHIRE AVE, SILVER SPRINGS, MD 20903  


DRAIN THE SWAMP OHIO
               tres_nm                                        address
14662        BARB COLE            102 RAINBOW CT, HILLSBORO

UNITED STATES SENATE JOINT ADVISORY COMMITTEE
       tres_nm                                         address
14026  JACK HU              1200 G ST NW, WASHINGTON, DC 20005
4106   JACK HU  1717 PENNSYLVANIA AVE NW, WASHINGTON, DC 20006


NATIONAL WILDLIFE FEDERATION ACTION FUND
                     tres_nm                                     address
16945  (no treasurer listed)  11100 WILDLIFE CENTER DR, RESTON, VA 20190
18186  (no treasurer listed)  11100 WILDLIFE CENTER DR, RESTON, VA 20190


AMERICA FIRST POLICIES INC.
                     tres_nm                                    address
17149  (no treasurer listed)       1400 CRYSTAL DR, ARLINGTON, VA 22202
18878  (no treasurer listed)  C/O BULLDOG COMPLIANCE, BEVERLY, MA 01915


CATALYST PAC
                tres_nm                            address
6544       CABELL HOBBS  PO BOX 1329, FORT MYERS, FL 33902
16720  LARISSA MARTINEZ   PO BOX 723234, ATLANTA, GA 31139


ANDRES FOR CONGRESS
                       tres_nm             

9705  EUGENE CRAIG            PO BOX 561, WHITE MARSH, MD 21162


CALIFORNIA PROFESSIONAL FIREFIGHTERS
                     tres_nm                                       address
17675            LOU PAULSON  1780 CREEKSIDE OAKS DR, SACRAMENTO, CA 95833
18997  (no treasurer listed)  1780 CREEKSIDE OAKS DR, SACRAMENTO, CA 95833


LAKE COUNTY REPUBLICAN PARTY
                 tres_nm                         address
10704        ANDY DUBOIS  PO BOX 1160, TAVARES, FL 32778
9410   THOMAS MONTGOMERY  PO BOX 1165, LUCERNE, CA 95458


SANDERS FOR CONGRESS
         tres_nm                                  address
14854  ERIC BLUM     398 CHESTNUT ST, CLAREMONT, NH 03743
9283   EVIE DIAZ  585 E16 ST BASEMENT, BROOKLYN, NY 11226


GROW OUR PARTY
                     tres_nm                                    address
16768       GEORGE HENNESSEY  23854 SHINNECOCK DR, SOUTH LYON, MI 48178
18336  (no treasurer listed)   2800 SHIRLINGTON RD, ARLINGTON, VA 22206


COOPER FOR CONGRESS
                 t

In [366]:
# show all cmte_nm & tres_nm for committees sharing a street address
multiadd = list(cm['address'].value_counts()[cm['address'].value_counts() > 10].index)

for add in multiadd:
    lil_cm = cm.loc[cm['address'] == add, ['tres_nm', 'cmte_nm']]\
    .sort_values(['tres_nm', 'cmte_nm'], ascending = [True, True]).reset_index(drop = True)
    print(add)
    print(lil_cm.to_csv(None))
    print('\n')

228 S WASHINGTON ST, ALEXANDRIA, VA 22314
,tres_nm,cmte_nm
0,DAVID SATTERFIELD,AMERICANS FOR LAW ENFORCEMENT PAC
1,DAVID SATTERFIELD,ARIZONA SUNRISE PAC
2,DAVID SATTERFIELD,ARKANSAS HORIZON
3,DAVID SATTERFIELD,BIGGS FOR CONGRESS
4,DAVID SATTERFIELD,BUILDING INNOVATIVE GOOD GOVERNMENT SOLUTIONS PAC
5,DAVID SATTERFIELD,CAROLINA PATRIOT ALLIANCE
6,DAVID SATTERFIELD,COMMITTEE TO PROTECT AMERICA'S ALLIES
7,DAVID SATTERFIELD,CONNECTICUT FIRST
8,DAVID SATTERFIELD,CONSERVATIVE RESULTS
9,DAVID SATTERFIELD,CONSTITUTIONAL CONSERVATIVES
10,DAVID SATTERFIELD,ELBERT GUILLORY'S AMERICA
11,DAVID SATTERFIELD,EXCELLENCE IN GOVERNMENT POLITICAL ACTION COMMITTEE
12,DAVID SATTERFIELD,FREE AT LAST PAC
13,DAVID SATTERFIELD,FREEDOM FUND FOR AMERICA'S FUTURE INC
14,DAVID SATTERFIELD,FRIENDS OF LENAR
15,DAVID SATTERFIELD,GRAIN AND FEED PAC
16,DAVID SATTERFIELD,GRASSROOTS AWARENESS PAC
17,DAVID SATTERFIELD,HOOSIER HORIZON
18,DAVID SATTERFIELD,IN09 VICTORY FUND
19,DAVID SATTERFIELD,JOHN CATS PAC
20,DAVID SATTERFI

918 PENNSYLVANIA AVE SE, WASHINGTON, DC 20003
,tres_nm,cmte_nm
0,AARON WATSON,BONNIE WATSON COLEMAN FOR CONGRESS
1,AMY ECKERT,BWC - BECAUSE WOMEN CAN PAC
2,ELLEN TAUSCHER,FEINSTEIN FOR SENATE 2024
3,ELLEN TAUSCHER,FUND FOR THE MAJORITY PAC
4,HALLE MAYES,GENERATION GO
5,JUDITH ZAMORE,2018 SENATE IMPACT
6,JUDITH ZAMORE,2020 COMMITTEE
7,JUDITH ZAMORE,AFTAB SWING LEFT FUND 2018
8,JUDITH ZAMORE,AMERICA WORKS FEDERAL PAC
9,JUDITH ZAMORE,AMERICA WORKS STATE & LOCAL PAC
10,JUDITH ZAMORE,ARIZONA NEVADA VICTORY FUND
11,JUDITH ZAMORE,ARIZONA OREGON VICTORY FUND
12,JUDITH ZAMORE,AVIATORS FOR ACTION VICTORY FUND
13,JUDITH ZAMORE,BATTLEGROUND VICTORY FUND 2018
14,JUDITH ZAMORE,BAY STATE FOR A MAJORITY
15,JUDITH ZAMORE,BETSY DIRKSEN LONDRIGAN VICTORY FUND
16,JUDITH ZAMORE,BLUE IL 18
17,JUDITH ZAMORE,BLUE SENATE 2018
18,JUDITH ZAMORE,BOOKER SENATE VICTORY
19,JUDITH ZAMORE,BROWN WARREN VICTORY FUND
20,JUDITH ZAMORE,CASEY WILD 2018
21,JUDITH ZAMORE,COMMON SENSE 2018
22,JUDITH ZAMORE,CONNECTICUT VICTORY 

2776 S ARLINGTON MILL DR, ARLINGTON, VA 22206
,tres_nm,cmte_nm
0,SCOTT MACKENZIE,BILL SPADEA FOR CONGRESS
1,SCOTT MACKENZIE,BLACK REPUBLICAN PAC
2,SCOTT MACKENZIE,CONSERVATIVE MAJORITY FUND
3,SCOTT MACKENZIE,CONSERVATIVE VICTORY FUND
4,SCOTT MACKENZIE,FOUNDED ON TRUTH
5,SCOTT MACKENZIE,HISPANICS FOR A CONSERVATIVE AMERICA
6,SCOTT MACKENZIE,MAN IN THE ARENA
7,SCOTT MACKENZIE,PEG LUKSIK FOR SENATE
8,SCOTT MACKENZIE,SAVE NEW YORK PAC
9,SCOTT MACKENZIE,STAND AMERICA PAC
10,SCOTT MACKENZIE,TEA PARTY MAJORITY FUND
11,SCOTT MACKENZIE,TEA PARTY VICTORY FUND
12,SCOTT MACKENZIE,THE CONSERVATIVE STRIKEFORCE
13,THEODORA JACKSON,JACKSON FOR VIRGINIA



2350 KERNER BLVD, SAN RAFAEL, CA 94901
,tres_nm,cmte_nm
0,(no treasurer listed),NEW MAJORITY CALIFORNIA
1,CHRISTOPHER SKINNELL,PERFORMANT FINANCIAL CORPORATION POLITICAL ACTION COMMITTEE
2,CHRISTOPHER SKINNELL,"PINNACLE ENTERTAINMENT, INC. PAC AKA PINNACLE ENTERTAINMENT PAC"
3,JASON KAUNE,BIOMARIN PHARMACEUTICAL INC. POLITICAL ACTION COMMITTEE AKA BI

### Other columns

In [367]:
cmtetp_dict = {
    'C' : 'cmte_CommunicationCost',
    'D' : 'cmte_DelegateCommittee',
    'E' : 'cmte_ElectioneeringComms',
    'H' : 'cmte_House',
    'I' : 'cmte_IndependentExpenditor',
    'S' : 'cmte_Senate',
    'N' : 'cmte_PACnonqual',
    'Q' : 'cmte_PACqual',
    'Y' : 'cmte_Partyqual',
    'O' : 'cmte_SuperPAC',
    'P' : 'cmte_Presidential',
    'U' : 'cmte_Singlecand',
    'V' : 'cmte_PACnoncontrib_nonqual',
    'W' : 'cmte_PACnoncontrib_qual',
    'X' : 'cmte_Partynonqual',
    np.nan : np.nan,
}

cm['cmte_tp'] = [cmtetp_dict[x] for x in cm['cmte_tp']]
cm['cmte_tp'].value_counts()

cmte_House                    4960
cmte_PACnonqual               3399
cmte_PACqual                  3250
cmte_SuperPAC                 2209
cmte_IndependentExpenditor    1326
cmte_Senate                   1001
cmte_Presidential              982
cmte_CommunicationCost         526
cmte_Partynonqual              299
cmte_PACnoncontrib_nonqual     286
cmte_ElectioneeringComms       285
cmte_Partyqual                 268
cmte_Singlecand                152
cmte_PACnoncontrib_qual         55
cmte_DelegateCommittee           5
Name: cmte_tp, dtype: int64

In [368]:
cmtedsgn_dict = {
    'A' : 'Authorized by a candidate',
    'B' : 'Lobbyist/Registrant PAC',
    'D' : 'Leadership PAC',
    'J' : 'Joint fundraiser',
    'P' : 'Principal campaign committee',
    'U' : 'Unauthorized',
    np.nan : np.nan,
}

cm['cmte_dsgn'] = [cmtedsgn_dict[x] for x in cm['cmte_dsgn']]
cm['cmte_dsgn'].value_counts()

Unauthorized                    9275
Principal campaign committee    6470
Lobbyist/Registrant PAC         1381
Joint fundraiser                 992
Leadership PAC                   666
Authorized by a candidate        217
Name: cmte_dsgn, dtype: int64

In [369]:
cm['cmte_pty_affiliation'].fillna('UNK', inplace = True)
cm.loc[cm['cmte_pty_affiliation'] == '.', 'cmte_pty_affiliation'] = 'UNK'
cm.loc[cm['cmte_pty_affiliation'] == 'UKN', 'cmte_pty_affiliation'] = 'UNK'
cm.loc[cm['cmte_pty_affiliation'] == '  0', 'cmte_pty_affiliation'] = 'UNK'
cm.loc[cm['cmte_pty_affiliation'] == 'UN', 'cmte_pty_affiliation'] = 'UNK'
cm['cmte_pty_affiliation'].value_counts()

UNK    11888
DEM     3000
REP     2668
IND      538
LIB      231
       ...  
ALP        1
J          1
WOR        1
WRI        1
S4A        1
Name: cmte_pty_affiliation, Length: 83, dtype: int64

In [370]:
cm['cmte_filing_freq'].value_counts()

Q    9007
A    5123
T    2943
M    1941
Name: cmte_filing_freq, dtype: int64

In [371]:
orgtp_dict = {
    'C' : 'org_Corporation',
    'H' : 'org_HnotonFECsite',
    'I' : 'org_InotonFECsite',
    'L' : 'org_Labor organization',
    'M' : 'org_Membership organization',
    'T' : 'org_Trade association',
    'V' : 'org_Cooperative',
    'W' : 'org_Corporation without capital stock',
    np.nan : np.nan,
}

cm['org_tp'] = [orgtp_dict[x] for x in cm['org_tp']]
cm['org_tp'].value_counts()

org_Corporation                          1744
org_Trade association                     757
org_Membership organization               529
org_Labor organization                    401
org_Corporation without capital stock     106
org_Cooperative                            45
org_HnotonFECsite                           7
org_InotonFECsite                           4
Name: org_tp, dtype: int64

In [372]:
cm['connected_org_nm'].value_counts()[:10]

NONE                                                                  3929
SERVE AMERICA VICTORY FUND                                              13
DOLLARS FOR DEMOCRATS                                                   12
REPUBLICANS INSPIRING SUCCESS & EMPOWERMENT PROJECT (RISE PROJECT)      11
DIGIDEMS COMMITTEE                                                       8
REPUBLICAN PARTY OF TEXAS                                                7
DEMOCRATIC PARTY OF VIRGINIA                                             6
REPUBLICAN PARTY OF VIRGINIA INC                                         6
REPUBLICAN NATIONAL COMMITTEE                                            6
FRESHMAN AGRICULTURAL REPUBLICAN MEMBERS TRUST AKA FARM TRUST            5
Name: connected_org_nm, dtype: int64

In [373]:
cm.loc[cm['connected_org_nm'] == 'NONE', 'connected_org_nm'] = np.nan

In [374]:
cm['connected_org_nm'].value_counts()[:10]

SERVE AMERICA VICTORY FUND                                            13
DOLLARS FOR DEMOCRATS                                                 12
REPUBLICANS INSPIRING SUCCESS & EMPOWERMENT PROJECT (RISE PROJECT)    11
DIGIDEMS COMMITTEE                                                     8
REPUBLICAN PARTY OF TEXAS                                              7
REPUBLICAN PARTY OF VIRGINIA INC                                       6
DEMOCRATIC PARTY OF VIRGINIA                                           6
REPUBLICAN NATIONAL COMMITTEE                                          6
CALIFORNIA REPUBLICAN PARTY                                            5
FRESHMAN AGRICULTURAL REPUBLICAN MEMBERS TRUST AKA FARM TRUST          5
Name: connected_org_nm, dtype: int64

In [375]:
# sorry! these print ugly but it's to display them unabbreviated
treses = list(cm['tres_nm'].value_counts()[cm['tres_nm'].value_counts() > 10].index)

for tres in treses:
    if tres != '(no treasurer listed)':
        lil_cm = cm[cm['tres_nm'] == tres]
        lil_cm = lil_cm[[
            'cmte_id',
            'cand_id',
            'cmte_nm',

        ]].sort_values(['cand_id', 'cmte_nm']).reset_index(drop = True)
        ptys = sorted(list(set(cm.loc[lil_cm.index, 'cmte_pty_affiliation'])))
        print(''.join([tres, ', treasurer']))
        print(len(lil_cm), 'committees')
        print('parties:', str(ptys))
        print(lil_cm.to_csv(None))
        print('\n\n')

PAUL KILGORE, treasurer
171 committees
parties: ['CRV', 'DEM', 'NAT', 'NNE', 'REP', 'UNK']
,cmte_id,cand_id,cmte_nm
0,C00569079,H0GA07125,HICE FREEDOM FUND
1,C00544445,H0GA07125,JODY HICE FOR CONGRESS
2,C00544510,H0GA08099,AUSTIN SCOTT VICTORY FUND
3,C00462556,H0GA09030,GRAVES FOR CONGRESS
4,C00501866,H0GA09030,TEAM GRAVES
5,C00458877,H0IL11052,KINZINGER FOR CONGRESS
6,C00467522,H0IL14080,RANDY HULTGREN FOR CONGRESS
7,C00461350,H0NH01217,FRIENDS OF FRANK GUINTA
8,C00476820,H0OH06189,JOHNSON FOR CONGRESS
9,C00497115,H0PA04220,ROTHFUS FOR CONGRESS
10,C00512236,H2AZ08102,MCSALLY FOR CONGRESS
11,C00504522,H2NC08185,HUDSON FOR CONGRESS
12,C00548818,H2NC08185,HUDSON FREEDOM FUND
13,C00364935,H2PA09035,BILL SHUSTER FOR CONGRESS
14,C00551374,H4AL06098,GARY PALMER FOR CONGRESS
15,C00573378,H4CO04090,BUCK FOR COLORADO
16,C00546846,H4FL26038,CARLOS CURBELO CONGRESS
17,C00565374,H4FL26038,CURBELO VICTORY COMMITTEE
18,C00543967,H4GA01039,BUDDY CARTER FOR CONGRESS
19,C00386755,H4GA06087,PRICE FOR CO

SHAWNDA DEANE, treasurer
20 committees
parties: ['DEM', 'NNE', 'REP', 'UNK']
,cmte_id,cand_id,cmte_nm
0,C00499822,H2CA03082,ALYSON HUBER CONGRESSIONAL EXPLORATORY COMMITTEE
1,C00639906,H8CA04186,ROCHELLE WILCOX FOR CONGRESS
2,C00638783,H8CA10118,DOTTY NYGARD FOR CONGRESS
3,C00649095,H8CA22147,FRANCO FOR CONGRESS
4,C00629774,H8CA34126,ALEJANDRA CAMPOVERDI FOR CONGRESS
5,C00667063,H8CA49116,CHRISTINA PREJEAN FOR CONGRESS
6,C00646794,,CA-BAM PAC
7,C00625848,,CALIFORNIA PROGRESS COALITION
8,C00621391,,CALIFORNIANS FOR OPPORTUNITY
9,C00501858,,EAST BAY YOUNG DEMOCRATS - FEDERAL
10,C00608471,,ELECTING WOMEN SILICON VALLEY FEDERAL PAC
11,C00641183,,FIGHT BACK CALIFORNIA
12,C00389189,,LAKE COUNTY DEMOCRATIC CENTRAL COMMITTEE
13,C70006028,,NATIONAL UNION OF HEALTHCARE WORKERS
14,C00658526,,NEED TO IMPEACH
15,C00572933,,PROGRESSIVE WOMEN SILICON VALLEY FEDERAL PAC
16,C00647628,,ROB BONTA FOR ASSEMBLY 2018
17,C00402826,,SAN DIEGO COUNTY DEMOCRATIC PARTY (FED. ACCT.)
18,C00529891,,SAN JOAQUIN COUN

In [376]:
cm.to_csv('data/03a_committees.csv', index = False)